In [ ]:
# container name
#cse6250-2018-11-10t02-36-55-285z
#storage_name
#flufylobster_69

In [1]:
%%info

In [1]:
%%configure -f 
{"name":"remotesparkmagics-sample", "executorMemory": "28G", "executorCores":6,"driverMemory":"28g", "heartbeatTimeoutInSecond":6000}

In [2]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
31,application_1541818620294_0035,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=remotesparkmagics-sample>

In [3]:
%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1541818620294_0011,pyspark,idle,Link,Link,✔


In [8]:
%%local
import os
os.getcwd()

'/var/lib/jupyter'

In [3]:
import pandas as pd


In [2]:
# textLines = spark.sparkContext.textFile('wasb:///example/data/gutenberg/ulysses.txt')

# textLines = spark.sparkContext.textFile('wasb:///ADMISSIONS.csv')

In [4]:
import pyspark.ml.feature 
from pyspark.ml.feature import CountVectorizer
import pyspark.sql.functions  as psf
from pyspark.sql import Window 
from pyspark.sql.types  import DateType
#import pandas as pd
#pd.set_option('display.max_colwidth', -1)
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType

In [5]:
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType
from pyspark.sql import SparkSession
from pyspark import SparkContext ,SparkConf

In [5]:
# wasb[s]://cse6250-2018-11-10t02-36-55-285z@flufy_lobster69.blob.core.windows.net/<path>

## Import data

In [5]:
admissions_sdf=(spark.read
    .option("inferschema", "true")
    .option("header", "true")
    .option("mode", "DROPMALFORMED")
    .csv("wasb:///ADMISSIONS.csv"))

In [46]:
# note_struct = StructType([StructField("ROW_ID", IntegerType(), True),
#                       StructField("SUBJECT_ID", IntegerType(), True),
#                       StructField("HADM_ID", StringType(), True),
#                       StructField("CHARTDATE", StringType(), True),
#                       StructField("CHARTTIME", StringType(), True),
#                       StructField("STORETIME", StringType(), True),
#                       StructField("CATEGORY", StringType(), True),
#                       StructField("DESCRIPTION", StringType(), True),
#                       StructField("CGID", StringType(), True),
#                       StructField("ISERROR", StringType(), True),
#                       StructField("TEXT", StringType(), True)])
notes_sdf=(spark.read
    .option("inferschema", "true")
    .option("header", "true")
    .option("delimiter", "|")
    .csv("wasb:///notes_discharge_pd.csv"))  

In [47]:
admissions_sdf.cache().take(1)

[Row(ROW_ID=21, SUBJECT_ID=22, HADM_ID=165315, ADMITTIME=datetime.datetime(2196, 4, 9, 12, 26), DISCHTIME=datetime.datetime(2196, 4, 10, 15, 54), DEATHTIME=None, ADMISSION_TYPE='EMERGENCY', ADMISSION_LOCATION='EMERGENCY ROOM ADMIT', DISCHARGE_LOCATION='DISC-TRAN CANCER/CHLDRN H', INSURANCE='Private', LANGUAGE=None, RELIGION='UNOBTAINABLE', MARITAL_STATUS='MARRIED', ETHNICITY='WHITE', EDREGTIME=datetime.datetime(2196, 4, 9, 10, 6), EDOUTTIME=datetime.datetime(2196, 4, 9, 13, 24), DIAGNOSIS='BENZODIAZEPINE OVERDOSE', HOSPITAL_EXPIRE_FLAG=0, HAS_CHARTEVENTS_DATA=1)]

In [9]:

#import notes 
# notes_df = spark.read.csv("wasb:///NOTEEVENTS.csv", header=True,schema=note_struct)
#csvFile = spark.read.csv('wasb:///HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv', header=True, inferSchema=True)

In [48]:
notes_sdf.cache().take(1)

[Row(SUBJECT_ID=22532, HADM_ID=167853.0, CATEGORY='Discharge summary', TEXT='Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]   Service: ADDENDUM:  RADIOLOGIC STUDIES:  Radiologic studies also included a chest CT, which confirmed cavitary lesions in the left lung apex consistent with infectious process/tuberculosis.  This also moderate-sized left pleural effusion.  HEAD CT:  Head CT showed no intracranial hemorrhage or mass effect, but old infarction consistent with past medical history.  ABDOMINAL CT:  Abdominal CT showed lesions of T10 and sacrum most likely secondary to osteoporosis. These can be followed by repeat imaging as an outpatient.                                [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D.  [**MD Number(1) 1776**]  Dictated By:[**Hospital 1807**] MEDQUIST36  D:  [**2151-8-5**]  12:11 T:  [**2151-8-5**]  12:21 JOB#:  [**Job Number 1808**] ')]

In [11]:
notes_sdf.dtypes, admissions_sdf.dtypes

([('SUBJECT_ID', 'int'), ('HADM_ID', 'double'), ('CATEGORY', 'string'), ('TEXT', 'string')], [('ROW_ID', 'int'), ('SUBJECT_ID', 'int'), ('HADM_ID', 'int'), ('ADMITTIME', 'timestamp'), ('DISCHTIME', 'timestamp'), ('DEATHTIME', 'timestamp'), ('ADMISSION_TYPE', 'string'), ('ADMISSION_LOCATION', 'string'), ('DISCHARGE_LOCATION', 'string'), ('INSURANCE', 'string'), ('LANGUAGE', 'string'), ('RELIGION', 'string'), ('MARITAL_STATUS', 'string'), ('ETHNICITY', 'string'), ('EDREGTIME', 'timestamp'), ('EDOUTTIME', 'timestamp'), ('DIAGNOSIS', 'string'), ('HOSPITAL_EXPIRE_FLAG', 'int'), ('HAS_CHARTEVENTS_DATA', 'int')])

## subset to used data

In [49]:
#create list of used columns 
adm_cols=['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',  'ADMISSION_TYPE']
notes_cols=['SUBJECT_ID', 'HADM_ID', 'CATEGORY','TEXT' ]
## subset notes and admissions to relavant data
adm_used= admissions_sdf[adm_cols]



notes_used = notes_sdf[notes_cols]
# change key type 
notes_used = notes_used.select('SUBJECT_ID', psf.col('HADM_ID').cast('int').alias('HADM_ID'), 'CATEGORY','TEXT' ).cache()
adm_used.printSchema()
notes_used.printSchema()

root
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ADMITTIME: timestamp (nullable = true)
 |-- DISCHTIME: timestamp (nullable = true)
 |-- DEATHTIME: timestamp (nullable = true)
 |-- ADMISSION_TYPE: string (nullable = true)

root
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- TEXT: string (nullable = true)

## Prepare admissions

In [50]:
sort_adm=adm_used.orderBy( psf.col("SUBJECT_ID"), psf.col('ADMITTIME'))
w = Window.partitionBy("SUBJECT_ID").orderBy(psf.desc('ADMITTIME'))
#next_adm= sort_adm.withColumn('next_admit', psf.lag(col('ADMITTIME')).over(w))
next_adm=sort_adm.select('*', psf.lag(psf.col('ADMITTIME')).over(w).alias('NEXT_ADMISSION')
                , psf.lag(psf.col('ADMISSION_TYPE')).over(w).alias('NEXT_ADMISSION_TYPE')).cache()

In [51]:
#example
next_adm\
.select('SUBJECT_ID','HADM_ID' ,'ADMITTIME' ,'NEXT_ADMISSION_TYPE','ADMISSION_TYPE','NEXT_ADMISSION')\
.where(psf.col("NEXT_ADMISSION").isNotNull())\
.show(2)

+----------+-------+-------------------+-------------------+--------------+-------------------+
|SUBJECT_ID|HADM_ID|          ADMITTIME|NEXT_ADMISSION_TYPE|ADMISSION_TYPE|     NEXT_ADMISSION|
+----------+-------+-------------------+-------------------+--------------+-------------------+
|      4101| 161951|2103-01-22 18:01:00|           ELECTIVE|     EMERGENCY|2106-09-17 11:45:00|
|      4900| 117413|2203-06-29 08:00:00|           ELECTIVE|      ELECTIVE|2203-10-26 13:15:00|
+----------+-------+-------------------+-------------------+--------------+-------------------+
only showing top 2 rows

### Add days to readmission

In [52]:
adm_datediff= next_adm.withColumn("days_next_admit", psf.datediff("NEXT_ADMISSION", "DISCHTIME")).cache()

In [53]:
##example :) 

adm_datediff.take(1)[0]['SUBJECT_ID']

148

## Prepare notes 

In [54]:
notes_used.select("CATEGORY").distinct().show()

+-----------------+
|         CATEGORY|
+-----------------+
|Discharge summary|
+-----------------+

In [55]:
## filter to discharge
#notes_dis_sum = notes_used.filter( notes_used.CATEGORY == 'Discharge summary')
## create index to dedup
w =  Window.partitionBy("SUBJECT_ID",'HADM_ID').orderBy(psf.desc('HADM_ID'))
notes_dis_sum  = notes_used.withColumn("row_number", psf.row_number().over(w)).cache()

In [56]:
## dedup
notes_dis_dedup=notes_dis_sum.filter(notes_dis_sum.row_number==1).cache()

In [57]:
notes_dis_sum.groupBy("SUBJECT_ID",'HADM_ID').count().filter(psf.col('count')>1).show(3)
notes_dis_dedup.groupBy("SUBJECT_ID",'HADM_ID').count().filter(psf.col('count')>1).show(3)


+----------+-------+-----+
|SUBJECT_ID|HADM_ID|count|
+----------+-------+-----+
|      1453| 122894|    2|
|      1616| 176986|    2|
|      2265| 100548|    2|
+----------+-------+-----+
only showing top 3 rows

+----------+-------+-----+
|SUBJECT_ID|HADM_ID|count|
+----------+-------+-----+
+----------+-------+-----+

In [58]:
# join notes and admisions remove newborns 
note_admit_join=adm_datediff\
.filter(psf.col('ADMISSION_TYPE')!='NEWBORN').join(notes_dis_dedup, ['HADM_ID','SUBJECT_ID'],'left').cache()

In [59]:
### check to see if cutoff of 30 is reasonable
sum_between= adm_datediff.select('days_next_admit').groupBy().sum().take(1)[0]['sum(days_next_admit)']
count_between= adm_datediff.select('days_next_admit').count()
sum_between/count_between

85.44102685838307

## create label 

In [60]:
#create label
note_admit_label=note_admit_join.select('*', psf.when(psf.col('days_next_admit')<30,1).otherwise(0).alias('label')).cache()

In [6]:

from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [23]:
# import string
# def tokenizer_better(text):
#     # tokenize the text by replacing punctuation and numbers with spaces and lowercase all words
#     if text != None:
#         punc_list = string.punctuation+'0123456789'
#         t = str.maketrans(dict.fromkeys(punc_list, " "))
#         text = text.lower().translate(t)
#         tokens = word_tokenize(text)
#     else:
#         tokens = []
#     return tokens
#spark_tokenizer = psf.udf(lambda text: tokenizer_better(text), ArrayType(StringType()))

## clean text

In [62]:
## to clean
import string
def text_cleaner(text):
    # tokenize the text by replacing punctuation and numbers with spaces and lowercase all words
    if text != None:
        punc_list = string.punctuation+'0123456789'
        t = str.maketrans(dict.fromkeys(punc_list, " "))
        text = text.lower().translate(t)
        #tokens = word_tokenize(text)
    else:
        text = ''
    return text
#register UDF

spark_cleaner= psf.udf(lambda text: text_cleaner(text),StringType())

# stopwords 

In [63]:
alphabet_low = []
for letter in range(97,123):
    alphabet_low.append(chr(letter))
stopwords=['','it', 'did', 'he', 'mustn', 'whom', "that'll", 'why', 'ain', "it's", 'isn', 'off', 'only', 'ourselves', 'which', 'to', 'she', 'during', "wouldn't", 'have', 'if', 'against', 'some', 'for', 'i', "don't", "didn't", 'ours', 'itself', 'should', 'very', 'those', 'him', 'just', 'll', "mustn't", 'out', 'theirs', 'each', 'needn', 'd', 'herself', 'few', 'himself', 'these', 'couldn', 'or', 'not', 'when', 'with', 'does', "needn't", "weren't", 'into', 'm', 'y', 'we', "wasn't", 'both', "doesn't", 'you', 'by', 'didn', 'our', 'was', 'wouldn', "haven't", 'more', 'who', 'same', 't', 'a', 'hadn', 'what', 'won', "couldn't", 'nor', 'they', 'shouldn', 'this', 'there', 'an', 'where', 'while', 'shan', 'has', 'yourself', 'so', 'had', 'myself', 'do', 'been', 'all', 'his', 'haven', 'between', "you've", 'doesn', 'at', 'me', 'then', 'until', 'above', 'ma', 'further', 'and', 'of', 're', 'o', 'on', 'up', 'its', 'her', 've', 'because', "isn't", 'than', "shan't", 'are', 'now', 'is', 'yours', 'will', 'be', 'mightn', 'their', 'being', 'about', 'your', "won't", 'don', "should've", 'wasn', 'how', 'am', 'weren', 'were', "hasn't", 'under', 'as', 'can', "you'd", 'from', 'once', "you'll", 'doing', 'too', 'down', "mightn't", 'before', 'themselves', "aren't", 'having', 'aren', 'no', 'over', 'them', 'through', 'but', "shouldn't", 'after', 's', 'hers', 'below', 'in', 'again', 'most', 'own', 'hasn', 'my', 'the', 'that', 'other', 'yourselves', 'such', 'any', "she's", 'here', "you're", "hadn't"]

In [71]:
idc9_word_list=['metallosis', 'attic', 'air', 'diarrhea', 'intrin', 'fumes', 'urticaria', 'tools', 'staphylococcal', 'valga', 'inborn', 'occ', 'delay', 'land', 'meckel', 'ricketts', 'myelopath', 'resid', 'upper', 'descemetocele', 'dizziness', 'legal', 'leu', 'mountain', 'caught', 'hypnotic', 'tic', 'full', 'nonmag', 'scrub', 'tobacco', 'uvula', 'integument', 'init', 'marine', 'serv', 'degree', 'chondritis', 'tooth', 'ophthalmoplegia', 'chylocele', 'esophagostomy', 'cardio', 'lymphprolif', 'cardc', 'my', 'psychogen', 'capillariasis', 'navicular', 'build', 'colostomy', 'amebiasis', 'weakness', 'skiing', 'incus', 'pinguecula', 'rickets', 'gonadal', 'nor', 'anurysm', 'disaccharidase', 'maxil', 'chromo', 'hyperplas', 'variabl', 'dancing', 'mother', 'dialy', 'exudative', 'valgus', 'deficit', 'obstruction', 'admin', 'image', 'paving', 'adenoids', 'inhib', 'attack', 'klinefelter', 'polio', 'dysrhythmia', 'kidny', 'miliary', 'luxation', 'aryepiglott', 'alcoh', 'tendinitis', 'constriction', 'irritability', 'ampulla', 'ill', 'circmpap', 'intermitt', 'compensatory', 'dmnt', 'monoarthrit', 'vomit', 'susp', 'neuroend', 'fing', 'recreation', 'loc', 'rct', 'vegtv', 'somatization', 'cheerleading', 'dfcile', 'agt', 'rflx', 'gestational', 'chg', 'aphthae', 'meno', 'expulsive', 'microglossia', 'exhaustion', 'mov', 'amntc', 'coronoid', 'ep', 'living', 'escalator', 'mosquit', 'thrombocytopathy', 'alcoholic', 'lateral', 'sputum', 'streptoc', 'phlycten', 'parst', 'atrophicae', 'wks', 'cellulocutaneous', 'contamination', 'fetal', 'mushrooms', 'danlos', 'applic', 'sight', 'sml', 'transver', 'interferon', 'extradur', 'prurigo', 'electrolyt', 'amphetamine', 'overact', 'ectopic', 'leishmaniasis', 'damage', 'elephantiasis', 'complet', 'ves', 'atresia', 'psychic', 'thoracot', 'subdural', 'adv', 'ulcer', 'igm', 'meningoenceph', 'cavrnosm', 'fem', 'priv', 'periton', 'obstetric', 'produce', 'superficial', 'argentous', 'gliding', 'synovitis', 'prsnl', 'overflow', 'chorioamnionitis', 'fm', 'lumbsacral', 'analeptics', 'promontor', 'subseq', 'transplant', 'imun', 'tympanosclerosis', 'fever', 'tissu', 'thlasema', 'growth', 'pyogen', 'monochr', 'cause', 'distres', 'diastl', 'antianemics', 'tympnoscler', 'tend', 'weightless', 'support', 'early', 'australian', 'wasp', 'antgn', 'vegetations', 'farm', 'mucosal', 'motorcycl', 'stereopsis', 'swelling', 'macrostomia', 'meat', 'sutures', 'endo', 'grft', 'mgmt', 'sacroiliac', 'osteoarth', 'lymp', 'rupture', 'craniofacial', 'jt', 'keloid', 'braziliense', 'facial', 'schizophr', 'subcap', 'electrocut', 'leukocytosis', 'neurosyphilis', 'exposure', 'dockers', 'longit', 'asympt', 'neonat', 'microphth', 'quills', 'immunity', 'suspct', 'resolv', 'paints', 'demyelination', 'extravasation', 'diffuse', 'pustular', 'perf', 'underweight', 'ovarn', 'digestv', 'immundef', 'hyper', 'matter', 'autonomic', 'hst', 'retrobulb', 'semicircul', 'echo', 'knife', 'crisis', 'lte', 'urethral', 'subac', 'class', 'sterilzation', 'bf', 'hypersecretion', 'retin', 'uterovag', 'prblm', 'phalanx', 'prim', 'porokrtsis', 'canaliculitis', 'rape', 'avalanche', 'conc', 'paresis', 'uncn', 'undescend', 'buttock', 'ileostomy', 'ancylostomiasis', 'vertigo', 'soap', 'hydroceph', 'illegitimate', 'elbow', 'th', 'zoonotic', 'housing', 'malposition', 'cd', 'chrn', 'macular', 'scr', 'postvaccination', 'qn', 'intst', 'intellct', 'root', 'brnch', 'lump', 'inflicted', 'neurohypophysis', 'plants', 'chlamydia', 'bevrag', 'clav', 'electr', 'cystourethrocele', 'replaced', 'depression', 'remain', 'eosinophilia', 'tip', 'rhy', 'incontnce', 'leukocyt', 'learning', 'hygiene', 'trichiasis', 'mitral', 'cr', 'symp', 'punc', 'srcoma', 'exudat', 'thoracogenic', 'cervicitis', 'schizoaffectv', 'sig', 'infestation', 'fev', 'necator', 'balantidiasis', 'palate', 'pers', 'plica', 'step', 'relationship', 'ecchymoses', 'swim', 'fungal', 'tourette', 'phacolytic', 'displacement', 'thnderclap', 'vagina', 'posttraum', 'analg', 'xtrndl', 'pstv', 'schizotypal', 'cell', 'texture', 'gend', 'hockey', 'scarring', 'tonsils', 'insipidus', 'frontal', 'facility', 'occp', 'oligohydramnios', 'impg', 'difficulty', 'testis', 'deple', 'typhus', 'amniot', 'elsewhere', 'grvs', 'sclerosing', 'keratin', 'vacuuming', 'hypertro', 'highway', 'thrpy', 'canalic', 'dematiacious', 'val', 'enterostomy', 'acetabuli', 'some', 'monteggia', 'elec', 'crsis', 'other', 'ebstein', 'dia', 'lag', 'smearing', 'expl', 'volvulus', 'tuberculoid', 'prem', 'tree', 'dressng', 'antenatl', 'vaccin', 'girdle', 'device', 'septl', 'conjoin', 'antin', 'breech', 'epiglottitis', 'electrocardiogram', 'synovit', 'incomptnce', 'tuboplasty', 'infundib', 'cavit', 'fibros', 'divergence', 'heterotopic', 'neck', 'neurasthenia', 'pterygium', 'water', 'adenopathy', 'myelodys', 'brawl', 'pulmonry', 'dermatoses', 'coaster', 'lab', 'unspf', 'manganese', 'urnry', 'pancreas', 'borne', 'acids', 'mod', 'corrosive', 'sarcoidosis', 'achlorhydria', 'pouchitis', 'poisoning', 'occlusal', 'juxtapap', 'antiasthmatics', 'meralgia', 'peri', 'corticoadren', 'epiphysiolysis', 'malignan', 'dissociated', 'icterohem', 'camp', 'hemoglobinuria', 'zygomycosis', 'bronchogenic', 'mningts', 'fugx', 'tmj', 'infantile', 'yrsnia', 'pigm', 'delirium', 'coal', 'cholesterin', 'plaque', 'hydroxyqu', 'lubricating', 'guidance', 'vit', 'vascular', 'incompetnt', 'firearms', 'newborn', 'bungee', 'minor', 'anaplastic', 'cutaneous', 'sus', 'intestinovesical', 'incontinence', 'retraction', 'ankle', 'postmastect', 'partial', 'syncyt', 'production', 'lupus', 'traum', 'sweat', 'extravstn', 'mucocele', 'ingest', 'fasc', 'thymus', 'mineral', 'feces', 'caries', 'puerperl', 'diabetes', 'misery', 'anaphyl', 'jakob', 'urethrocele', 'cease', 'tu', 'symptomatc', 'endos', 'nonobstet', 'porphyrin', 'bronchitis', 'follwg', 'whoop', 'craft', 'urinary', 'vessels', 'toxin', 'dacryoadenitis', 'menorrhagia', 'takayasu', 'epiphy', 'cave', 'spcf', 'caruncle', 'orient', 'rider', 'sym', 'curve', 'polyneuritis', 'orthopnea', 'obstruct', 'syndrom', 'gr', 'schmorl', 'regional', 'canal', 'balance', 'chemistry', 'rhabdomyolysis', 'engorge', 'refsum', 'panniculitis', 'atony', 'spherocytosis', 'glues', 'uran', 'symptm', 'factitia', 'thorac', 'rea', 'circulatory', 'benzdiaz', 'chld', 'level', 'prosthetic', 'power', 'intervention', 'sinoatrial', 'stomac', 'train', 'methemoglobinemia', 'myopthy', 'chemothr', 'degenerative', 'endotrach', 'ocular', 'elderly', 'paroxyml', 'partner', 'emesis', 'touch', 'with', 'hypertn', 'diet', 't', 'paral', 'acte', 'sex', 'eructat', 'travel', 'prtoneum', 'alveolitis', 'granulations', 'blg', 'slp', 'infus', 'choriomening', 'mysthna', 'vescouretrl', 'infected', 'cataracta', 'optc', 'gstritis', 'lacrim', 'advrs', 'infertil', 'diast', 'ischiocapsular', 'super', 'rapid', 'disl', 'chills', 'deutan', 'wall', 'vrtb', 'vasc', 'mgr', 'thrt', 'cholinergics', 'herbicides', 'metacarpophalan', 'sublingual', 'aneurysm', 'psychological', 'reduc', 'charge', 'sperm', 'cervicofac', 'hypostasis', 'noncv', 'metabolism', 'pneumon', 'fibrinolysis', 'distrb', 'complete', 'stabbing', 'aura', 'sublux', 'antacids', 'vacnation', 'palpitations', 'play', 'synovial', 'accessory', 'caregiv', 'one', 'enterococcus', 'rgdt', 'cough', 'myopath', 'schizoaffectve', 'entrcltca', 'subst', 'outlet', 'reye', 'glucose', 'gain', 'const', 'hyperpla', 'derang', 'responders', 'squamous', 'gambian', 'height', 'zoophilia', 'somatic', 'leukodystrophy', 'saginata', 'immunodefic', 'osteoarthr', 'flexure', 'acq', 'osteopoikilosis', 'secondary', 'alteration', 'antepar', 'bowling', 'precordial', 'hypotony', 'aftrcr', 'periph', 'gamma', 'salt', 'suffocat', 'ganglion', 'coli', 'inst', 'cestode', 'gen', 'pertussis', 'rough', 'polyostotic', 'neoplas', 'await', 'vapor', 'derv', 'metabolic', 'hyperestrogenism', 'quinoline', 'authority', 'mandb', 'winds', 'trophoblas', 'type', 'otitic', 'aseptic', 'exert', 'bipo', 'choroidal', 'gastrt', 'chnge', 'neopl', 'no', 'prsn', 'robertson', 'penile', 'hpros', 'seizures', 'fibula', 'nervousness', 'restrct', 'amniotc', 'clsr', 'defor', 'ductus', 'margnl', 'galactorrhea', 'east', 'enc', 'adequate', 'chrnc', 'civilian', 'multigrav', 'episclerit', 'hypergammaglobu', 'conjunctivitis', 'arenaviral', 'ileum', 'dysphagia', 'stripprs', 'elliptocytosis', 'lichen', 'psychotropic', 'effus', 'allrgic', 'grnd', 'cth', 'home', 'sg', 'tunic', 'coagultion', 'psychomotor', 'gonococcal', 'prpherl', 'interst', 'nonmed', 'iv', 'autonom', 'thyrotoxic', 'surfactant', 'martial', 'nsrt', 'hallucinogen', 'kyphosis', 'symptomatic', 'keratit', 'frag', 'trauma', 'syndrome', 'succinimides', 'ara', 'rickettsialpox', 'vallecula', 'mastopathy', 'malg', 'zone', 'unspec', 'enzy', 'monoamn', 'horseplay', 'burns', 'tonsillitis', 'marg', 'mand', 'suf', 'multnod', 'conjoined', 'flncy', 'opiates', 'papillae', 'angiitis', 'antineop', 'sudn', 'guided', 'kid', 'min', 'herpeticum', 'tonsillar', 'encopresis', 'innervation', 'rash', 'abdom', 'solvents', 'polyclon', 'nitroglycol', 'sulfur', 'boutonniere', 'nonhosp', 'feigning', 'nonobl', 'antibiot', 'puerprl', 'lyme', 'descending', 'impairment', 'hornet', 'gases', 'muilatn', 'septic', 'gravs', 'corticosteroids', 'venom', 'cholangitis', 'ulcers', 'surfce', 'antepart', 'pyrimid', 'gma', 'unattended', 'mower', 'undetrmine', 'compl', 'area', 'capillary', 'erythematous', 'phosphorus', 'chagas', 'fixation', 'pyrazole', 'ascorbic', 'postphl', 'membranes', 'rubra', 'prod', 'hpt', 'for', 'aircrf', 'celr', 'expos', 'cyanides', 'dfbrltr', 'oxidation', 'smr', 'gummata', 'choanal', 'hydrocele', 'all', 'ctr', 'lumbosacr', 'schwannomatosis', 'prepatellar', 'neurotroph', 'viruses', 'brucellosis', 'define', 'invol', 'dstl', 'deviated', 'meningococcus', 'dvce', 'qt', 'dentl', 'oral', 'coronary', 'muscular', 'epilep', 'dysfnc', 'smallpox', 'vert', 'explos', 'incndiary', 'dig', 'hypertrophic', 'faucial', 'orthopoxvirus', 'instru', 'simple', 'lb', 'surveillance', 'hydronephrosis', 'ulc', 'legs', 'sparganosis', 'addv', 'ataxia', 'destruct', 'pedestrian', 'nvl', 'running', 'respir', 'duodenitis', 'hypertens', 'fld', 'nasolacrml', 'preglaucoma', 'spotting', 'iliac', 'phthirus', 'artery', 'carrier', 'product', 'presentat', 'myelodysplastic', 'serotn', 'amines', 'motor', 'pedophilia', 'multiloc', 'nonflaccid', 'golf', 'well', 'schizoaffective', 'dependen', 'globe', 'glaucomatocyclit', 'radiotherapy', 'preparation', 'glutathione', 'studies', 'multigravida', 'intrapelv', 'immat', 'choroideremia', 'involv', 'bicornuate', 'insomn', 'normal', 'env', 'firearm', 'priapism', 'disturbance', 'myoneural', 'stevens', 'rhinoscleroma', 'wrestling', 'descrep', 'persisting', 'postablativ', 'main', 'unicornuate', 'les', 'ss', 'interarch', 'nonabsorb', 'endocervix', 'stiffness', 'war', 'scanty', 'app', 'cervx', 'longitudinal', 'autoimmune', 'sensonrl', 'retina', 'vitamin', 'keratopathy', 'disr', 'papyraceous', 'reasons', 'pseudocyst', 'pharyngeal', 'parasitic', 'low', 'telangiec', 'gc', 'correspond', 'mlt', 'mansoni', 'alkalosis', 'testic', 'metatarsophal', 'aircr', 'parasit', 'skul', 'landscaping', 'afric', 'atlg', 'othr', 'osteomalacia', 'glandular', 'atrphy', 'puncture', 'chlor', 'edema', 'else', 'vaginal', 'enterit', 'unstable', 'polyps', 'maternal', 'postpartum', 'rake', 'uncnr', 'occup', 'boat', 'fib', 'obturator', 'endocrin', 'rickettsioses', 'infarct', 'birth', 'four', 'glaucoma', 'bloom', 'measles', 'otoscler', 'degr', 'abnormality', 'drop', 'ten', 'iden', 'mass', 'perichondritis', 'hemarthrosis', 'drain', 'convulsions', 'paracoccidioidomycosis', 'fitting', 'stroke', 'intraepithl', 'epiphora', 'ligament', 'asthma', 'sealpox', 'mlg', 'fstr', 'veins', 'sensations', 'hypothyr', 'aqueous', 'dgstv', 'traf', 'spermatic', 'lesser', 'prol', 'merkel', 'tetanic', 'aerobic', 'bypas', 'nral', 'aerobics', 'system', 'confusional', 'superf', 'hyp', 'attemp', 'within', 'hypoplasia', 'ef', 'broke', 'movements', 'repl', 'enteros', 'dependent', 'respiratry', 'bmi', 'nigra', 'err', 'perfor', 'malignancy', 'hypermature', 'heterotrop', 'nr', 'nonconv', 'und', 'onst', 'antinfct', 'testicular', 'thyroidit', 'swimmer', 'extrahepat', 'trigone', 'ant', 'reaction', 'transp', 'bee', 'mycoplasma', 'ach', 'obst', 'jug', 'sclr', 'lymphangitis', 'pulp', 'trm', 'narrow', 'monocytic', 'peripartum', 'maintain', 'urethritis', 'macrolid', 'uncntr', 'postn', 'diving', 'morb', 'vers', 'pseudomemb', 'comb', 'vib', 'sensation', 'androgen', 'depr', 'cardiac', 'posteruptv', 'anoml', 'apnea', 'mild', 'endoc', 'tuberculosis', 'currnt', 'dyskinesia', 'scarlet', 'polyarthrit', 'equinovarus', 'aftcr', 'intraspinal', 'fingers', 'chemical', 'endocarditis', 'curvature', 'hemododialysis', 'variant', 'myelop', 'tympanoscler', 'remiss', 'postpa', 'physical', 'tracheoesophageal', 'procreative', 'glycogen', 'injur', 'dir', 'prox', 'hernia', 'pathogen', 'bil', 'cystocele', 'cathetr', 'lymphoma', 'pathological', 'neurolog', 'coagul', 'neurosis', 'abortion', 'dual', 'hypox', 'above', 'immunodef', 'biologic', 'mesenteric', 'antiviral', 'grade', 'consln', 'chro', 'com', 'crn', 'strangulat', 'melioidosis', 'microgenia', 'pos', 'multisystem', 'elvtd', 'drip', 'inert', 'disuse', 'retracted', 'cosmetics', 'phlbts', 'sequela', 'vs', 'reticular', 'cluster', 'prolpse', 'hairy', 'twin', 'curr', 'ceylanicum', 'ped', 'snorkel', 'empyema', 'req', 'labyrin', 'dysphsia', 'tetrach', 'infantum', 'antiprotoz', 'sympatholytics', 'dplt', 'umbl', 'leprosy', 'infl', 'angl', 'retic', 'hypert', 'kennedy', 'cramps', 'orthodontic', 'subdur', 'rifle', 'poxvirus', 'quartan', 'seeds', 'vscourtl', 'abandonment', 'encephalit', 'transmission', 'adverse', 'body', 'malignant', 'coccidioidal', 'dystrph', 'amino', 'chlamydiae', 'intrauterine', 'angioid', 'psngr', 'blepharitis', 'anest', 'embryonic', 'orb', 'alvelr', 'bruxism', 'peyronie', 'nutr', 'sacrotuberous', 'respiratn', 'episcl', 'syphilit', 'dysarthria', 'condyloma', 'sacrospinatus', 'parachutist', 'polyhydramnios', 'prph', 'plant', 'riding', 'mechanism', 'vasodilator', 'hydrat', 'enzyme', 'sicca', 'tremens', 'hemophil', 'pyrophosph', 'nas', 'pacemaker', 'univrsl', 'hypovent', 'unknw', 'catarac', 'noxious', 'househld', 'intramural', 'louse', 'opposition', 'leukoencep', 'resident', 'arbovirus', 'manifest', 'buccal', 'thrush', 'mycobacterial', 'membran', 'diplacusis', 'antiperson', 'hypoact', 'trapezoid', 'thicknes', 'pres', 'spf', 'ml', 'gastric', 'plasmacytosis', 'restoratn', 'prfnd', 'hep', 'chiari', 'radioact', 'hamartomas', 'cholelithias', 'anticoagul', 'mastocytoma', 'inflamtry', 'condyle', 'cholesteatosis', 'sphinctr', 'stnd', 'glaucma', 'abnorml', 'row', 'flrq', 'emphysematous', 'unstagebl', 'polycystic', 'maternl', 'protein', 'buphthalmos', 'postlaminectomy', 'sclp', 'flamethrower', 'vitamins', 'hyperemesis', 'frisbee', 'papillitis', 'contraindication', 'diffus', 'acidifying', 'nephropathy', 'endomet', 'actinomycosis', 'cris', 'tube', 'episod', 'iud', 'neuromuscle', 'retclsrc', 'instr', 'quarry', 'specf', 'crnry', 'silicone', 'exanthemata', 'lattice', 'hypodermic', 'hi', 'critical', 'sebaceous', 'hidden', 'snse', 'typ', 'apathy', 'soft', 'vaccine', 'enterovirus', 'artificial', 'subaortic', 'launch', 'subcutan', 'raft', 'anatomical', 'mort', 'kayk', 'sharp', 'haematobium', 'coag', 'vertebral', 'cardiovascular', 'encephlalo', 'leukem', 'atopic', 'investigate', 'monoclon', 'diphther', 'floor', 'lymphangioma', 'parals', 'hypnot', 'trampoline', 'appendix', 'pvc', 'ciliary', 'phantom', 'breath', 'opiat', 'myalgia', 'iii', 'current', 'bowman', 'hydrocephal', 'lym', 'pruritic', 'athrsc', 'react', 'inflnz', 'staphylcocc', 'bleeding', 'cysticercosis', 'caisson', 'bariatric', 'malform', 'fiber', 'unsocial', 'antigastric', 'nonplaque', 'seabase', 'physicl', 'coagulants', 'viremia', 'asth', 'postrheum', 'periocular', 'vaccines', 'glanders', 'neurologic', 'atrioven', 'bypass', 'gyn', 'diuretic', 'pelv', 'bronchiectas', 'circadian', 'football', 'mplga', 'anoxia', 'sulfonamides', 'enamel', 'depletion', 'impair', 'excess', 'resis', 'insufficienc', 'treatment', 'stomch', 'lfstyle', 'steroid', 'metatarsus', 'bph', 'reiter', 'uncert', 'retardation', 'monoart', 'pleuris', 'med', 'tetracycline', 'ups', 'broncholitis', 'abuse', 'regular', 'hantavirus', 'watercraft', 'hmrhg', 'pacemake', 'sialoadenitis', 'care', 'aftcare', 'polymyalgia', 'incompl', 'contracture', 'var', 'infect', 'bacteruria', 'medial', 'trichostrongyliasis', 'uncer', 'malleus', 'unaggress', 'embolus', 'absces', 'pilates', 'polydipsia', 'quadruplet', 'osteoporosis', 'anaph', 'anphylct', 'talus', 'expressive', 'embl', 'bursitis', 'fusel', 'fther', 'halluc', 'progressive', 'strabism', 'pseudobulbar', 'extract', 'boxing', 'parnchym', 'perineocele', 'trau', 'varices', 'nonhyperact', 'postinfusion', 'marasmus', 'lymphocytosis', 'weapon', 'encntr', 'septal', 'nspcf', 'equine', 'ears', 'cardiorsp', 'vomiting', 'nonnarc', 'juv', 'infertility', 'lymphosarc', 'ureterocele', 'serratia', 'secr', 'submers', 'perineal', 'antisocial', 'f', 'ulcrtion', 'feared', 'acuminatum', 'chondrocalc', 'pre', 'lysis', 'spondylos', 'imperforate', 'idiopth', 'hmrg', 'sign', 'eczem', 'chorioretinal', 'genu', 'wiskott', 'isoimmunization', 'pack', 'meningit', 'fetopelv', 'inf', 'lymphsrc', 'obsrv', 'cerebel', 'deg', 'floppy', 'circ', 'gingiv', 'acromioclavic', 'shared', 'boiler', 'monplga', 'splen', 'disloc', 'xtrcorp', 'mitochondrial', 'rectocele', 'supv', 'absence', 'carbon', 'eight', 'orthopedic', 'pancreat', 'optic', 'saliv', 'eating', 'eosinophil', 'indcd', 'bif', 'allergens', 'active', 'ladder', 'hemoglobinopathies', 'lftsded', 'prader', 'brief', 'parxysmal', 'oris', 'perip', 'supraglottis', 'nonmotor', 'harada', 'midcervic', 'dentofac', 'hypertonicity', 'parasitism', 'tract', 'lght', 'syphilitic', 'vst', 'stage', 'po', 'pleural', 'reason', 'preop', 'separation', 'intertrochanteric', 'peanuts', 'pilonidal', 'adult', 'aftr', 'gft', 'heavy', 'fam', 'disulfi', 'strangul', 'bronchus', 'spotted', 'nasal', 'periostitis', 'bundle', 'execution', 'trophoblastic', 'nonpyogenic', 'perinatal', 'rod', 'episdic', 'contin', 'region', 'fussy', 'mucopur', 'accid', 'junct', 'osteolysis', 'fngr', 'nonparaly', 'ischium', 'hesitancy', 'external', 'methylphendate', 'prune', 'antemet', 'norepinephrn', 'spr', 'derangement', 'edentulsm', 'placent', 'casts', 'stomach', 'stg', 'acidosis', 'meta', 'brass', 'prelab', 'radiohumeral', 'cleanup', 'sexual', 'artry', 'chair', 'compounds', 'angle', 'hemophilia', 'srvl', 'ganglia', 'capitis', 'insuff', 'strp', 'whipple', 'episcleritis', 'contraction', 'valve', 'cholestma', 'transport', 'overxrt', 'bell', 'intract', 'suppress', 'compr', 'sbdrml', 'trac', 'paintball', 'slow', 'obsessive', 'knee', 'clinical', 'letterer', 'lumbosacral', 'atherosclerosis', 'dich', 'mucositis', 'rem', 'achromatopsia', 'periumbilic', 'opacit', 'adenoviral', 'anomaly', 'devic', 'voice', 'enuresis', 'swallowing', 'color', 'uncontrold', 'willi', 'antitussives', 'fmily', 'fascioliasis', 'sbac', 'osteoarthro', 'amputat', 'tx', 'occipitpost', 'remsion', 'interocclusl', 'hematoma', 'systemic', 'metablsm', 'recurvatum', 'true', 'myasthna', 'brain', 'urine', 'granulation', 'rhinosporidiosis', 'rubber', 'steroids', 'rhythm', 'pancreatic', 'pseudopapilledema', 'correct', 'edentulism', 'xanthelasma', 'mood', 'papillomavirus', 'nonsupp', 'septicem', 'amp', 'drgrst', 'immunosuppres', 'atheroscler', 'metacarpus', 'bb', 'enterocolitis', 'nontox', 'subchronic', 'affect', 'obliterans', 'soccer', 'instrument', 'equip', 'chlm', 'murmurs', 'wheelchair', 'pneumococcal', 'maculopathy', 'adiposity', 'nfct', 'hyphema', 'periodic', 'initiate', 'versicolor', 'gestat', 'schizophrenia', 'veneral', 'dog', 'cma', 'surgcl', 'incomp', 'sem', 'eros', 'progress', 'zonular', 'anxiety', 'ground', 'ureter', 'metaplas', 'cutting', 'temporal', 'lig', 'croup', 'hemicran', 'drmtit', 'prescript', 'identity', 'hyperten', 'fasting', 'woodworking', 'chemicals', 'isotope', 'aggres', 'foot', 'hist', 'recent', 'prelabor', 'islets', 'megacolon', 'deposits', 'db', 'nerve', 'frontotemp', 'basal', 'chromos', 'present', 'vogt', 'exceptionally', 'abdmnal', 'skates', 'live', 'probl', 'parasomnia', 'insct', 'misaln', 'mastodynia', 'prolng', 'behv', 'inguinale', 'act', 'l', 'mutism', 'vestibulitis', 'nonvenom', 'hgb', 'solitary', 'uninod', 'second', 'supp', 'exfol', 'veg', 'extraction', 'des', 'adaptat', 'young', 'puerp', 'vrl', 'beats', 'falciparum', 'psychot', 'cytomegalovirus', 'cath', 'anallrg', 'quad', 'confirmation', 'fox', 'insomnia', 'narclpsy', 'uncntrl', 'dicalc', 'acromioclavicular', 'inferopost', 'staphylococcus', 'vehic', 'uteri', 'lasers', 'cs', 'pathways', 'artic', 'locked', 'legionnaires', 'cadmium', 'discomfort', 'hyperparathyrd', 'dvrtcli', 'punctate', 'explosions', 'piano', 'vibratory', 'rsp', 'lacera', 'hosp', 'delusion', 'indeterminate', 'vaccinia', 'psoriatic', 'premat', 'mutilatn', 'butyrophenone', 'deletions', 'fol', 'accidnt', 'certif', 'hyprtrophc', 'crd', 'eval', 'contract', 'female', 'incipient', 'cereb', 'gingivitis', 'visuospatial', 'withdrawal', 'talipes', 'furniture', 'film', 'degen', 'cryptosporidiosis', 'motcycl', 'repetit', 'boutonneuse', 'block', 'metabol', 'cholelithiasis', 'digging', 'deplete', 'rh', 'malpres', 'phenylketonuria', 'hordeolum', 'lary', 'agent', 'hodgkin', 'hemiblock', 'pneumoconiosis', 'glucrtcod', 'copper', 'anmly', 'df', 'sched', 'childhood', 'building', 'hormon', 'antibody', 'hyperhidrosis', 'psorias', 'parasympathomimetic', 'cheek', 'hyperparathyroid', 'arteriosus', 'eyelids', 'reticulosarcoma', 'freon', 'femur', 'aspiration', 'intraeph', 'ruptur', 'below', 'dx', 'membr', 'five', 'dsct', 'ultraviol', 'delivrd', 'free', 'mammogram', 'ventricle', 'fu', 'sys', 'choledocholithiasis', 'amnion', 'facilities', 'tranq', 'media', 'vein', 'injuries', 'atrophic', 'cardiovas', 'fistu', 'insnsitvty', 'hemoperitoneum', 'rel', 'complicata', 'convert', 'sanguinous', 'choroiditis', 'wound', 'disintegr', 'turbinat', 'obsv', 'plicated', 'hypermetropia', 'schizophreniform', 'lacrm', 'diuretics', 'intermed', 'urin', 'lmb', 'chemo', 'dfct', 'autosom', 'aromat', 'referral', 'pill', 'perph', 'hole', 'mouth', 'ankylosing', 'between', 'gangosa', 'uncontrld', 'circumscri', 'bimalleolar', 'norml', 'pneumn', 'asymptomat', 'extrinsic', 'instrum', 'sborheic', 'hmiplga', 'oil', 'caust', 'encph', 'hypermobility', 'japanese', 'abnormal', 'tarsometatars', 'hypochondriasis', 'pit', 'hemophagocytic', 'torticollis', 'typh', 'drown', 'perinat', 'postauricular', 'innomin', 'cent', 'disorders', 'constit', 'psyc', 'mutation', 'cntrl', 'periradicular', 'ritter', 'entangl', 'nonvenomous', 'kwashiorkor', 'fibromatosis', 'nodosa', 'myelody', 'gangr', 'bran', 'kidney', 'unil', 'nut', 'shortn', 'pyelonephritis', 'total', 'uric', 'nongc', 'traumat', 'conjun', 'postinf', 'xeroderma', 'compnd', 'earthquakes', 'plexus', 'immersion', 'sarcoma', 'infectn', 'endothel', 'profnd', 'malpos', 'gastroschisis', 'degeneration', 'nonsp', 'passage', 'malleolus', 'lobomycosis', 'intrcran', 'natve', 'intrcaps', 'thlassemia', 'monocular', 'deliver', 'carbn', 'metacarpophalang', 'colic', 'basketball', 'hiking', 'cerebell', 'fusion', 'intox', 'monitor', 'gross', 'fatty', 'ossicle', 'unspecf', 'hypersens', 'explosn', 'cerebvasc', 'passive', 'excessive', 'blastomycosis', 'urinostomy', 'stc', 'retrobulbar', 'degenerat', 'mines', 'onset', 'mot', 'bronch', 'monox', 'undescended', 'prevent', 'extravag', 'nodule', 'tech', 'blunt', 'meas', 'rsk', 'hyposmolality', 'grand', 'ans', 'recess', 'fructose', 'hdgk', 'gf', 'malfun', 'lac', 'infantl', 'conv', 'physio', 'hypothermia', 'clubbing', 'symptom', 'floods', 'toxicologic', 'microstomia', 'ozzardi', 'digit', 'custody', 'terrorism', 'recurr', 'closure', 'neurlgcl', 'movmt', 'fertlity', 'sacr', 'cardioresp', 'antitrypsin', 'menstrual', 'wo', 'cannabis', 'detach', 'bbb', 'parasite', 'subapo', 'pole', 'colonic', 'coloboma', 'jumping', 'hered', 'anuria', 'klebsiella', 'desc', 'carbam', 'new', 'bubonic', 'alcoholism', 'movement', 'noise', 'paratyphoid', 'bone', 'simp', 'hemorr', 'cylinder', 'finding', 'kyphoscoliotic', 'coxa', 'cecum', 'pain', 'thrombocyt', 'intraocular', 'diabetic', 'idiopathc', 'alexia', 'cry', 'pericardial', 'rabies', 'rick', 'episdc', 'trimalleolar', 'stat', 'bacter', 'oxygen', 'dermatographic', 'dysrhythmias', 'attacks', 'replace', 'swellng', 'syndr', 'sept', 'fet', 'disch', 'langerhan', 'tricyclc', 'cathartic', 'overweight', 'leptospiros', 'chafeensis', 'secundum', 'percept', 'mediastinum', 'pneumonitis', 'mxd', 'spinocerebellar', 'nonunion', 'proc', 'mastect', 'menin', 'asept', 'varicella', 'orophryn', 'toxoplasm', 'weather', 'onchocerciasis', 'bunion', 'claw', 'chat', 'nonparalyt', 'unilt', 'conception', 'bruch', 'chromosom', 'types', 'disseminated', 'splenomegaly', 'fnd', 'febrile', 'instit', 'glucagon', 'functional', 'congenita', 'ttrcycln', 'fuel', 'depres', 'commissure', 'om', 'gs', 'collapse', 'proteinosis', 'rmson', 'fragments', 'mucopolysaccharidosis', 'chondrosternal', 'mast', 'h', 'subclav', 'skateboard', 'hepatopulmonary', 'grm', 'plaq', 'emotional', 'ureteroves', 'specfd', 'disruption', 'nt', 'antirheumatics', 'candidal', 'cochlear', 'uremic', 'alter', 'overload', 'old', 'streptobacillary', 'sulph', 'antimony', 'ster', 'moderate', 'leukorrhea', 'antibio', 'def', 'lymphoid', 'sore', 'jts', 'sense', 'asthmat', 'crystl', 'oophoritis', 'exten', 'neoplastic', 'abdomen', 'macroglossia', 'postcholecystectomy', 'comm', 'enzymes', 'colon', 'recur', 'change', 'replacement', 'adjust', 'defec', 'sulfnmides', 'end', 'choledocholith', 'hourglass', 'gastritis', 'proctitis', 'disability', 'pigmentation', 'diamniotc', 'develop', 'erysipelas', 'diabet', 'antibiotic', 'cradle', 'parent', 'intellect', 'delivered', 'insemination', 'cryptophthalmos', 'dyschromia', 'false', 'atheroembolism', 'pallor', 'environment', 'radiat', 'subglottis', 'posthemorrhag', 'stridor', 'calcul', 'prion', 'eff', 'keratos', 'longitudin', 'flecks', 'syndactyly', 'cerebrosp', 'cuff', 'pedicul', 'neuralgic', 'sac', 'spondylolisthesis', 'essen', 'hem', 'wear', 'factitious', 'horseback', 'antineopl', 'lymphocyt', 'renal', 'poison', 'transt', 'giant', 'asymp', 'symblepharon', 'extrm', 'hypofunction', 'craniorachischisis', 'encephlomy', 'cord', 'animal', 'anemia', 'car', 'asexual', 'suspect', 'hallucin', 'trigeminal', 'preeclamp', 'meningitis', 'obj', 'rd', 'tubrcloma', 'mno', 'tricuspid', 'eent', 'subserous', 'cleaning', 'stiffnes', 'prctosigmoidtis', 'antimetabol', 'lt', 'esophageal', 'cushion', 'dubois', 'rat', 'margins', 'hazards', 'petrol', 'brderln', 'trigger', 'rubeosis', 'myocarditis', 'cho', 'schistosoma', 'object', 'adolesc', 'consciousnes', 'cirrhosis', 'mucosa', 'bathing', 'macrogenia', 'paraproteinemia', 'oliguria', 'hammer', 'urethra', 'rappell', 'palliative', 'wrist', 'milit', 'adlt', 'animals', 'late', 'centric', 'ice', 'tackle', 'mechan', 'sclerema', 'ft', 'tendinae', 'drugs', 'retnoph', 'victim', 'prolong', 'chalazion', 'fuch', 'thrombocythemia', 'burkitt', 'immatur', 'stillborn', 'postphleb', 'intr', 'vermilion', 'kpsi', 'coccyx', 'antagonist', 'occupant', 'pattrn', 'midgut', 'maj', 'vertical', 'famly', 'diaphragmatic', 'histologic', 'transfusn', 'sde', 'intrusion', 'a', 'endoscopy', 'unspcf', 'heredit', 'tap', 'immunotherapy', 'natv', 'hormones', 'psoriasis', 'radioulna', 'gallbladder', 'astrovirus', 'hypract', 'extrac', 'strptcs', 'cholesteatoma', 'collat', 'murine', 'grass', 'embryoic', 'persn', 'mcsl', 'metl', 'prolapsed', 'disease', 'sedative', 'weiss', 'dna', 'dest', 'vlt', 'trypanosomias', 'intestine', 'grth', 'hydrocarb', 'aldrich', 'primum', 'ichthyosis', 'brawny', 'resprtry', 'stool', 'myringitis', 'six', 'orthodontics', 'ear', 'maltreatment', 'disturb', 'work', 'liveborn', 'sponge', 'stoma', 'fat', 'pursuit', 'bones', 'cerebrovasc', 'hyperthermia', 'hetrochrom', 'farmers', 'keto', 'dracontiasis', 'occlsl', 'nodul', 'familial', 'pulm', 'trspl', 'condition', 'enter', 'aftrcare', 'forarm', 'medicolegal', 'orthostatic', 'extadural', 'altrn', 'evans', 'pharynx', 'echoencephalogram', 'dors', 'tissue', 'larynx', 'peripart', 'rev', 'smoke', 'ptosis', 'dacryops', 'fd', 'hypoxc', 'night', 'paresthetica', 'subcaps', 'exc', 'submucosal', 'vascul', 'endophthal', 'effect', 'dyston', 'psychogenic', 'developmental', 'oxytocic', 'kernicterus', 'incompet', 'temp', 'corticobasal', 'anesthet', 'laryngeal', 'pneumocon', 'speech', 'carditis', 'dgst', 'extrem', 'lipodystrophy', 'gingival', 'fertility', 'embols', 'stimul', 'emphysema', 'rad', 'imnprfl', 'atrph', 'parametritis', 'hypoten', 'varicose', 'angular', 'segmnt', 'symbolic', 'sero', 'problems', 'pleur', 'capsul', 'monoamniotc', 'coll', 'synov', 'gastrostomy', 'underwater', 'shuddering', 'minimal', 'nb', 'torus', 'inhaled', 'blister', 'suppurat', 'milking', 'horseshoe', 'chloral', 'therml', 'dissem', 'fordyce', 'hypercoagulable', 'hypoplsia', 'cardiomyopath', 'nontrauma', 'ovulation', 'equinus', 'iridocyclit', 'osteopetrosis', 'ridden', 'space', 'respirator', 'decubitus', 'hyperpotassemia', 'driv', 'globulin', 'social', 'prx', 'comp', 'biloculare', 'pilaris', 'anserinus', 'elevated', 'inhibitors', 'insuf', 'bradycardia', 'ind', 'airc', 'brest', 'unhappiness', 'staining', 'glucocorticoid', 'allergic', 'secondry', 'methicln', 'metals', 'blk', 'purpura', 'shld', 'enterocol', 'state', 'forgn', 'planus', 'neural', 'closed', 'nodes', 'accident', 'parietal', 'antconvl', 'phthisical', 'organism', 'magnet', 'velo', 'trans', 'microdeletions', 'stimulnt', 'malgnt', 'streptococcal', 'ftng', 'cystoid', 'lup', 'gerstmn', 'double', 'lunate', 'mnplg', 'tab', 'aphakia', 'prinzmetal', 'levine', 'reduction', 'poliomyelit', 'acquire', 'ndr', 'miosis', 'cyst', 'dioxide', 'vrnt', 'pyoderma', 'vertiginous', 'pneumonopathy', 'oculomotor', 'ulceration', 'corrosiv', 'prob', 'exces', 'manic', 'desire', 'spasm', 'rch', 'recruitment', 'detrusr', 'szry', 'nspf', 'hyperosmolality', 'buc', 'childhd', 'spondylolysis', 'hydrate', 'menisc', 'lown', 'submer', 'spstc', 'iatrogenc', 'visceral', 'vulvovaginitis', 'beriberi', 'thigh', 'repetv', 'malnutr', 'noninfl', 'cholinergic', 'dental', 'canth', 'cuboid', 'scapula', 'fleischer', 'y', 'into', 'punctum', 'history', 'vasculariza', 'enteroviral', 'retrograde', 'osteoar', 'spc', 'cyclist', 'sites', 'abo', 'vc', 'emancipation', 'eczema', 'incomplt', 'crater', 'vaginl', 'sialolithiasis', 'hemophthalmos', 'td', 'postvaricella', 'myelodyspls', 'bilat', 'endophthalmia', 'lordosis', 'sternum', 'af', 'sidewalk', 'somatoform', 'triatriatum', 'pervasv', 'kugelberg', 'aftercre', 'black', 'premise', 'arthrit', 'hypocalcem', 'vesicos', 'phimosis', 'myel', 'retained', 'autonmc', 'solium', 'anhydra', 'illeg', 'descend', 'conval', 'pp', 'cyclophoria', 'sideroblastic', 'budd', 'pull', 'redun', 'pneumogastric', 'inhal', 'sympt', 'generalized', 'histo', 'suffocation', 'cosmetic', 'accretions', 'genital', 'tietze', 'saliva', 'hypomanic', 'respsys', 'masochism', 'asian', 'iridodialysis', 'dispro', 'takotsubo', 'mold', 'txn', 'radius', 'lo', 'stm', 'rheumatoid', 'hurricane', 'ventricular', 'redness', 'irreg', 'mntl', 'infrc', 'swimming', 'amnestc', 'surgical', 'ileocolitis', 'nonteratogenic', 'asym', 'adlscnt', 'pigmnt', 'pyromania', 'formation', 'common', 'spond', 'myotonic', 'nucl', 'ven', 'aftrcre', 'perin', 'dialysis', 'butyrphen', 'cervix', 'suprf', 'hypopharynx', 'dissociative', 'bariatrc', 'substance', 'exostosis', 'vesical', 'ectodermal', 'cauliflower', 'detergent', 'cyclothymic', 'donor', 'srg', 'spcfd', 'ventric', 'catheter', 'sliding', 'dust', 'human', 'preterm', 'basilar', 'ah', 'wnd', 'o', 'altitude', 'paragang', 'avoid', 'calisthenics', 'stomatits', 'pemphigoid', 'pas', 'fall', 'nonbac', 'pericardit', 'vulvodynia', 'labor', 'utility', 'derangment', 'dementia', 'conjunctiv', 'crowd', 'algoneurodystrophy', 'incomplete', 'hyperkinet', 'disem', 'gigantism', 'infarction', 'conduct', 'goit', 'dfibrillator', 'postablat', 'alchl', 'refraction', 'rickettsial', 'oropharynx', 'myoglobinuria', 'thyrotoxicosis', 'therap', 'maxilla', 'neuron', 'syph', 'plex', 'admissn', 'mammographic', 'scald', 'antipsychotc', 'deformity', 'radiofreq', 'meniere', 'lacriml', 'rx', 'acute', 'cytomegaloviral', 'myocardit', 'dficncy', 'medicinal', 'ent', 'lead', 'neuralgia', 'adherent', 'astigmatism', 'infrct', 'feb', 'reg', 'term', 'leak', 'liq', 'interphaln', 'nonpsychot', 'pharyngoesoph', 'contu', 'compulsive', 'scaffolding', 'computer', 'overexert', 'bltrl', 'anesth', 'doubling', 'pseudomonas', 'unwanted', 'vestibular', 'spondyl', 'mrsa', 'telogen', 'mallory', 'conjuc', 'mediterranean', 'flat', 'vault', 'summer', 'stenos', 'arthrop', 'docker', 'residence', 'hemophilus', 'hemolyt', 'herpetic', 'relief', 'movmnt', 'grandparent', 'vasodilators', 'medicin', 'sd', 'dialys', 'influ', 'phenothiazine', 'gastrc', 'calcaneus', 'cocaine', 'candidiasis', 'throat', 'ben', 'prstate', 'anphyl', 'endophthalm', 'encountr', 'fusn', 'dysenter', 'bird', 'sympathomimetics', 'language', 'extern', 'fourth', 'peritonsillar', 'nonpsych', 'tenosynovitis', 'submandibular', 'mn', 'cerb', 'coccidiosis', 'divert', 'periprosthetc', 'necess', 'warts', 'humer', 'toes', 'misadventure', 'dentine', 'opiate', 'labyrinthine', 'vertebra', 'flail', 'satiety', 'perfrm', 'panophthalmitis', 'dvt', 'days', 'venous', 'phar', 'infan', 'radiological', 'pharmaceut', 'mach', 'dsrp', 'graf', 'dsrdr', 'incor', 'brncpul', 'per', 'tumor', 'straus', 'bacterial', 'peroxisomal', 'effct', 'rsv', 'resp', 'backache', 'elevation', 'damag', 'crys', 'relapsing', 'overanxious', 'atrial', 'cystitis', 'retinit', 'angiopathy', 'retnd', 'osteodystrophy', 'meningismus', 'follic', 'unsatis', 'natur', 'noncmplnt', 'spine', 'carpal', 'neovasculizat', 'dyskinesa', 'salmonella', 'clonorchiasis', 'finish', 'appendico', 'internl', 'unpow', 'unk', 'adnexa', 'obesity', 'nervosa', 'primary', 'pityriasis', 'vivax', 'fire', 'finger', 'osteitis', 'pylorospasm', 'indeterm', 'paran', 'nondysent', 'ltr', 'cauda', 'psoas', 'echinococ', 'swimmers', 'anesthesia', 'anus', 'exudates', 'dysthymic', 'cerebvas', 'curren', 'hcg', 'infraspinatus', 'bandemia', 'cen', 'ob', 'percussion', 'myelm', 'struck', 'lpd', 'dribbling', 'endometrial', 'perirenal', 'urogenital', 'fasciitis', 'amphetamin', 'incompat', 'bronchiectasis', 'behav', 'amnglcsds', 'homosexual', 'cholecys', 'lactation', 'heterophoria', 'elb', 'retard', 'ventilation', 'interscplr', 'nonprolf', 'sibling', 'pericard', 'fibrosclerosis', 'entangle', 'styloid', 'grt', 'intes', 'localized', 'wandering', 'tox', 'employ', 'odontogenic', 'npht', 'suff', 'mch', 'occupan', 'res', 'nasopharynx', 'neo', 'idiopathic', 'hebephrenia', 'deformans', 'dmi', 'unconven', 'maintenance', 'cancrum', 'hydromyelia', 'hpat', 'arthrodesis', 'thrombocytopenia', 'chromoblastomycosis', 'rotavirus', 'cardiomyopathy', 'pois', 'blood', 'rheum', 'pharyngitis', 'psych', 'relaps', 'margin', 'reading', 'ascariasis', 'suprficial', 'termination', 'sft', 'herpesvr', 'alrgc', 'monoam', 'snd', 'chorea', 'shovel', 'ornithosis', 'orbt', 'opac', 'contusion', 'mushroom', 'port', 'plateau', 'exanthema', 'tear', 'periodontitis', 'mvmt', 'strep', 'tinea', 'bilirubin', 'lens', 'inj', 'gstrt', 'hydrocyan', 'sea', 'inadv', 'arterioven', 'celiac', 'fx', 'hemplg', 'aircraft', 'postpart', 'counsel', 'antithyroid', 'nodular', 'prolif', 'myogenic', 'funct', 'postcricoid', 'stem', 'own', 'antbiot', 'maladjustment', 'rotator', 'blackwater', 'demoralization', 'red', 'spermatocele', 'cervicobrachial', 'isolation', 'pregnt', 'descemet', 'cupping', 'excavatum', 'agrcult', 'musculocutan', 'agitans', 'passeng', 'capsule', 'alight', 'diphyllobothrias', 'objects', 'sing', 'indust', 'trigem', 'pls', 'baking', 'pneum', 'sclerosis', 'hypertrichosis', 'pulpitis', 'contrac', 'bacteremia', 'trochlear', 'mal', 'hpv', 'iridoschisis', 'skull', 'entropion', 'testing', 'antimycobac', 'flag', 'neuroleptic', 'stmp', 'keratomalacia', 'edwards', 'ret', 'erylk', 'phosph', 'nephr', 'lacrosse', 'need', 'lipoid', 'cryptococcosis', 'acid', 'helicobacter', 'art', 'polycyst', 'enterocoltis', 'medicine', 'beach', 'ovarian', 'penetr', 'fnctnl', 'trochanteric', 'musculosk', 'bony', 'medullary', 'xtrrndl', 'semen', 'chromosome', 'tophi', 'amnesia', 'erosion', 'follow', 'congestion', 'dates', 'pedes', 'deterrent', 'stone', 'sewing', 'request', 'erth', 'infective', 'meth', 'cholecystitis', 'analges', 'furnit', 'aplastc', 'constitution', 'lung', 'failed', 'venereum', 'tympanic', 'exacer', 'mvmnts', 'keratoconjunctivit', 'bi', 'parox', 'hyperglyceridemia', 'callosities', 'butyrophen', 'behcet', 'hyprpl', 'deprivation', 'pstntl', 'subcut', 'overdos', 'exp', 'subtot', 'brkt', 'mvmnt', 'mycot', 'purpra', 'hodgkins', 'ptca', 'off', 'spec', 'methyl', 'breast', 'shyness', 'popliteal', 'postmeasles', 'local', 'climb', 'corporis', 'unp', 'chronc', 'langs', 'digital', 'infest', 'rr', 'cardia', 'contents', 'encount', 'gingititis', 'intestinl', 'men', 'adnex', 'calicivirs', 'antiplte', 'noninf', 'seafood', 'tisue', 'persist', 'psycho', 'gait', 'func', 'iliofemoral', 'track', 'perivent', 'residenc', 'organ', 'prostatitis', 'gouty', 'calculus', 'gastrointestinal', 'migrans', 'flood', 'undeterm', 'insertion', 'orgsm', 'secret', 'retropharyngeal', 'person', 'transfsn', 'sphincter', 'digest', 'suffocate', 'mining', 'asbestos', 'transplt', 'lid', 'mesenteritis', 'discrim', 'alv', 'induction', 'swan', 'epididymis', 'hypogastric', 'daily', 'auditory', 'delliv', 'ctrct', 'dwarfism', 'sciatic', 'atroph', 'st', 'spcrft', 'malfunction', 'eyelid', 'wndw', 'lower', 'penetrat', 'oth', 'inner', 'infections', 'insert', 'shape', 'flccd', 'stern', 'puer', 'yellow', 'rdiat', 'dis', 'fallopian', 'ovale', 'scleritis', 'lymphadenitis', 'site', 'diphtheria', 'base', 'bile', 'saccadic', 'intravag', 'any', 'embol', 'occlusion', 'intravn', 'myoclonus', 'convalescence', 'atax', 'intermit', 'gran', 'prfile', 'counslng', 'dysmetabolic', 'siderosis', 'manfest', 'regulat', 'infiltr', 'androgens', 'patau', 'psychostimulant', 'ribs', 'dural', 'radicular', 'cyclosporiasis', 'psychos', 'aromatase', 'prphyl', 'scoliosis', 'majora', 'brill', 'oculogram', 'exhibitionism', 'crush', 'astringent', 'conjunctiva', 'epidrml', 'necros', 'microcheilia', 'mix', 'rmsn', 'sensormotr', 'elder', 'buphthal', 'bact', 'disrupt', 'assoc', 'deploy', 'great', 'undetermin', 'marrow', 'group', 'antiallrg', 'reentrant', 'fetus', 'diseas', 'vulvar', 'horm', 'steal', 'hyperfunc', 'chrm', 'befor', 'interphalan', 'nursemaid', 'anticoagulants', 'interstitl', 'anxiolyt', 'nk', 'psychosexual', 'cartilage', 'anastom', 'adpt', 'piped', 'pyrophos', 'nec', 'modrate', 'findings', 'suprv', 'preserv', 'laboratory', 'plastc', 'exac', 'ht', 'prepuce', 'mesent', 'abnormalities', 'presbyopia', 'striking', 'fungicides', 'pemphigus', 'income', 'coracoclavicular', 'orifice', 'tracheal', 'separ', 'oblit', 'lability', 'phobia', 'mallet', 'antigen', 'trnk', 'nondm', 'inflict', 'refereeing', 'vibration', 'con', 'conversion', 'to', 'dependence', 'whole', 'hypoplas', 'ironing', 'specif', 'varus', 'economic', 'alkal', 'struct', 'compartment', 'inflamed', 'instrmnt', 'shaped', 'sealant', 'drum', 'echinococc', 'acanthamoeba', 'ehlers', 'herpetiform', 'acetazolamide', 'blindness', 'postv', 'microaneurysm', 'migrne', 'nd', 'longitud', 'disfigurement', 'zygomatic', 'musc', 'transfusion', 'shotgun', 'galactosemia', 'bomb', 'histry', 'glycosuria', 'anomalous', 'auto', 'suppur', 'binocular', 'sten', 'laceration', 'immune', 'cavities', 'gasoline', 'lie', 'back', 'circumstances', 'unilateral', 'iib', 'cochlvestib', 'psng', 'neg', 'side', 'rosacea', 'lesions', 'cortical', 'digestive', 'dermatomyositis', 'aplasia', 'acariasis', 'threaten', 'hyprprthyrd', 'plasmacytoma', 'oligospermia', 'seroma', 'pica', 'pyomyositis', 'thrombocyto', 'adn', 'aur', 'eversion', 'tb', 'fatal', 'cerebral', 'hemoph', 'crises', 'profound', 'spacing', 'shaft', 'fibrillation', 'supernumerary', 'chiasm', 'nondom', 'mercury', 'antiemet', 'agricult', 'situ', 'acanthosis', 'cryst', 'tuberculoma', 'cerv', 'trifascicular', 'cerebr', 'threatened', 'movemnt', 'submucous', 'mesh', 'dwel', 'serms', 'plague', 'perfrin', 'anhidrosis', 'dysfunction', 'hemivertebra', 'luteum', 'cavus', 'blanca', 'psychophysic', 'pyrexia', 'frgmt', 'induct', 'construction', 'dome', 'anticholinerg', 'calf', 'virus', 'papill', 'misplaced', 'wiring', 'cholesteatma', 'intrau', 'hypertropia', 'otit', 'abnrml', 'lbr', 'heartburn', 'domestic', 'mole', 'artficial', 'waldeyer', 'pneu', 'olecranon', 'carinatum', 'absnce', 'nondmnt', 'cog', 'hepatic', 'north', 'rhodesian', 'parapoxvirus', 'femoral', 'manc', 'tr', 'deposit', 'irradiation', 'azoospermia', 'cut', 'mandible', 'powr', 'athscl', 'repr', 'pasteurellosis', 'midline', 'cnv', 'septicemia', 'vermil', 'count', 'heparin', 'streaks', 'hemrrhoid', 'cerebe', 'hypercalcemia', 'chamber', 'isoimmunizat', 'streptococus', 'intra', 'laryngotrach', 'smoking', 'arsenic', 'hdache', 'tubal', 'arthus', 'isolated', 'scleroderma', 'on', 'anesthetic', 'strang', 'monochorion', 'viabil', 'ath', 'varice', 'antianemic', 'hydrops', 'specified', 'ab', 'obsc', 'dyshormonogenic', 'coccidioidomycos', 'contour', 'paragran', 'mil', 'narcolepsy', 'painful', 'nongnccl', 'pyeloureteritis', 'pneumococcus', 'placenta', 'patulous', 'dysfun', 'antmycob', 'chondrocalcin', 'duplication', 'neuropathy', 'myeloid', 'healed', 'psychotic', 'dysfunc', 'neop', 'goniosynechiae', 'v', 'dt', 'uncomplic', 'calcaneal', 'cntrcp', 'staph', 'vitreous', 'tetracyclc', 'gonorrhea', 'religion', 'esophagitis', 'ducts', 'hunger', 'laundry', 'hematpoitc', 'spacecraft', 'behavior', 'radiouln', 'hyperem', 'dwell', 'outer', 'valv', 'mumps', 'cerebellum', 'cardiovsclr', 'noninfect', 'exfoliation', 'encounter', 'pseudomonal', 'psychosis', 'epiglottis', 'appendicitis', 'rom', 'rheumatica', 'caps', 'discharge', 'matern', 'vsc', 'shrt', 'encephmye', 'berries', 'tympanum', 'sympathetic', 'prickly', 'glans', 'crnitne', 'hematospermia', 'pyloric', 'mantle', 'rodenticides', 'mydrtc', 'vrs', 'reuptake', 'basophilia', 'lactam', 'pat', 'parkinson', 'renl', 'recurnt', 'exposur', 'phase', 'exacerb', 'boogie', 'heterotropia', 'pleura', 'econo', 'dystocia', 'postprocedural', 'pylori', 'contracted', 'petrositis', 'combined', 'corneal', 'hyperkeratosis', 'spastic', 'brncio', 'surf', 'rickettsiosis', 'generat', 'abnor', 'trtmt', 'ied', 'flu', 'anat', 'snow', 'salping', 'erythrmia', 'container', 'synovium', 'prostate', 'demen', 'mocycl', 'aromatc', 'cumltv', 'crstacns', 'centipede', 'ulnar', 'regul', 'contagiosum', 'structure', 'hern', 'synd', 'unspecified', 'webbing', 'tenosynov', 'invers', 'bronchiect', 'psychiatric', 'articulation', 'megaloblastic', 'tenonitis', 'fetalis', 'swallowed', 'dec', 'non', 'death', 'stress', 'lipoma', 'circulat', 'paralysis', 'fmly', 'crbl', 'beh', 'deplet', 'clo', 'microtia', 'narc', 'granulom', 'epistaxis', 'enophth', 'dissociatve', 'hematologic', 'atriovent', 'myelitis', 'metastat', 'aggression', 'cri', 'vd', 'concus', 'traff', 'stndng', 'endocard', 'irritable', 'over', 'lft', 'manifestations', 'hhv', 'ventltr', 'cytol', 'listeriosis', 'dfbrl', 'labyrinthitis', 'spch', 'mottled', 'adrenergics', 'hemorrhage', 'tens', 'puberty', 'condensans', 'brn', 'and', 'utro', 'parathyroids', 'devices', 'scleromalacia', 'supracondyl', 'palindrom', 'paranoid', 'ca', 'pancyt', 'keratoconjunc', 'dermatomycosis', 'muscskel', 'dudn', 'intervent', 'defncy', 'depressn', 'limbar', 'eruption', 'dystonic', 'insulin', 'dehydration', 'wat', 'backgrnd', 'gland', 'orthotic', 'serous', 'myocardial', 'reactive', 'subj', 'environ', 'aphonia', 'overhang', 'helminth', 'cruciate', 'microscopic', 'dsrd', 'medic', 'neuronitis', 'corac', 'esophag', 'feeding', 'trnspl', 'papillary', 'genetic', 'meninges', 'perforat', 'muc', 'collision', 'explosives', 'torsion', 'african', 'g', 'nonspeaking', 'mcycl', 'episodc', 'constrictiv', 'postural', 'refuse', 'syndromes', 'lumbr', 'defiant', 'general', 'lagophthalmos', 'koyanagi', 'relx', 'changes', 'bldder', 'prp', 'clos', 'healthy', 'pleurisy', 'vir', 'otorrhea', 'variations', 'crcinoid', 'circul', 'assault', 'move', 'carcinoid', 'cyanosis', 'thrive', 'emotnl', 'circumstance', 'internal', 'academic', 'dysphonia', 'aid', 'major', 'ms', 'blepharochalasis', 'tributary', 'enmy', 'transloc', 'tornado', 'endophthalmitis', 'dacryocystitis', 'anteprtm', 'penicillin', 'mononucleosis', 'dsrders', 'micropenis', 'hallucinations', 'muscle', 'urinry', 'retrolental', 'communication', 'hyperchylomicronemia', 'emetics', 'vitreomacular', 'bacteriuria', 'allrg', 'coagulation', 'adenoiditis', 'drills', 'filariasis', 'thiamine', 'food', 'co', 'essence', 'explosiv', 'continuous', 'acrom', 'endocr', 'adhesive', 'lang', 'crf', 'shga', 'dys', 'parathyroid', 'neovasculariz', 'trichuriasis', 'thrmbtc', 'fstula', 'status', 'antidepresnt', 'communic', 'peripheral', 'streptococcus', 'plane', 'previa', 'therapeutic', 'grde', 'pel', 'sat', 'strom', 'enceph', 'laryngitis', 'encephalomyel', 'foregut', 'infra', 'passge', 'wheezing', 'encephal', 'forceps', 'somat', 'autologs', 'chondromalacia', 'cavovarus', 'postnasal', 'berry', 'illg', 'pigment', 'multiphasic', 'sinus', 'rtctl', 'unintentl', 'demul', 'face', 'sixth', 'impairm', 'presbyacusis', 'nystagmus', 'multi', 'conjugate', 'twins', 'angioneurotic', 'diastematomyelia', 'crimean', 'dyshidrosis', 'schizophrenic', 'underfill', 'methadone', 'falling', 'ovaries', 'anticoag', 'bartonellosis', 'vitiligo', 'iia', 'dysreflexia', 'remn', 'thrombos', 'bodies', 'epitheliopathy', 'extrapyramidal', 'third', 'nonatlg', 'thoracoabd', 'without', 'psgr', 'usa', 'hanging', 'reproduct', 'engorgemnt', 'coordination', 'focal', 'gum', 'cls', 'rheumatism', 'chorioretinitis', 'anon', 'pneumothorax', 'dyslexia', 'livebrn', 'stpmth', 'tests', 'cardiorespirat', 'botulism', 'disrpt', 'coarct', 'node', 'plas', 'mycplsm', 'cytomeg', 'lymphogranuloma', 'communicat', 'pesticide', 'vaginismus', 'erythromyc', 'mortar', 'pulmonale', 'malocc', 'sciatica', 'ntrutr', 'dlt', 'palsy', 'adol', 'prst', 'kissing', 'hyponasality', 'gardening', 'phlebotomus', 'lethal', 'perioculr', 'nausea', 'hemolytc', 'mening', 'bct', 'urtroplv', 'thoracoscop', 'ankyl', 'variola', 'hirschsprung', 'colosty', 'prt', 'posttraumatic', 'chain', 'schizoafftv', 'tubr', 'proln', 'transaminase', 'parasympatholytics', 'anal', 'hydrceph', 'urea', 'handgun', 'vasodil', 'fragm', 'uns', 'ossifican', 'mstoid', 'lymphomas', 'stiff', 'pleurodynia', 'exos', 'scrotal', 'preg', 'acd', 'alveol', 'pneumni', 'keratoconjunctivitis', 'br', 'genitourin', 'xerosis', 'mi', 'antivaricose', 'anhydrotic', 'team', 'dmii', 'dyspl', 'electrolyte', 'opportunistic', 'cor', 'batter', 'grwth', 'shigellosis', 'obstructiv', 'tmp', 'ang', 'septate', 'mathematics', 'sacral', 'leucocoria', 'spouse', 'burning', 'sensorneur', 'hamate', 'glen', 'obstet', 'internum', 'palmar', 'hypothrm', 'congenital', 'dermatitis', 'chancroid', 'ex', 'mutilate', 'explosive', 'roseola', 'reattach', 'pedal', 'beck', 'cntrcep', 'menstl', 'uniltrl', 'retroperitoneum', 'fibroelastosis', 'gas', 'oval', 'jnc', 'hereditary', 'clot', 'assist', 'plsm', 'drmtitis', 'ego', 'involun', 'morbid', 'inapprt', 'leg', 'hypofun', 'adnx', 'unsp', 'tmr', 'station', 'histoplasmosis', 'histrionic', 'calcer', 'inapp', 'mobitz', 'mutilation', 'derange', 'eustach', 'myl', 'cyclitis', 'serotinin', 'emblsm', 'methaqualone', 'thyroid', 'athletics', 'idio', 'clavicle', 'pseudoexfoliat', 'anaphylactic', 'dietetics', 'tachycardia', 'mucopurul', 'dermatos', 'carpometacarp', 'abdomn', 'erythromelalgia', 'fetishism', 'sylvatic', 'granulomatosis', 'stn', 'cardiomegaly', 'gun', 'ulcr', 'myopathies', 'orgasmic', 'bilateral', 'fluid', 'blad', 'spondylopathy', 'sheath', 'hemochromatos', 'fragment', 'window', 'devel', 'hypovolemia', 'achalasia', 'perit', 'erupt', 'clft', 'chemotherapy', 'contact', 'bipolor', 'interpersonal', 'uni', 'nonauto', 'paroxysml', 'orig', 'stapes', 'psychstimulant', 'onychia', 'stenosis', 'collisn', 'abort', 'enteric', 'recp', 'nonexudat', 'fetl', 'antidepress', 'els', 'adjustmnt', 'undiff', 'rectum', 'meniscus', 'pancytopenia', 'meds', 'polyart', 'neoplasm', 'nts', 'disordrs', 'louis', 'strabismus', 'minora', 'rhyth', 'warfare', 'holiday', 'canl', 'menst', 'otosclerosis', 'tachycard', 'hypothyroid', 'multinodul', 'tripl', 'incisional', 'piercing', 'sensory', 'subchr', 'anerobes', 'occupational', 'defect', 'hemoptysis', 'mech', 'phlegmon', 'tuberos', 'prophylact', 'neurit', 'pneumocystosis', 'eaton', 'equina', 'arteritis', 'impl', 'gal', 'asphyxiation', 'thrmb', 'auricle', 'nod', 'crowding', 'hyprtrophy', 'epithelium', 'staphylma', 'panenceph', 'cap', 'infarc', 'pyelonephr', 'antprotazoal', 'lacerat', 'abrasion', 'du', 'retractile', 'affective', 'varioliformis', 'nonpsychotic', 'walking', 'smell', 'schizoph', 'joints', 'detachment', 'felon', 'hypotrichosis', 'tymp', 'lifting', 'gene', 'willebrand', 'not', 'hebephren', 'aircrft', 'hydroureter', 'cert', 'hemrg', 'pneumonit', 'dystonia', 'hot', 'deficient', 'process', 'truncus', 'overfill', 'laser', 'iridocyclitis', 'rugby', 'recurrent', 'arrest', 'subcondylar', 'hypertension', 'conduction', 'chem', 'gastroent', 'weight', 'calc', 'poisn', 'neutropenic', 'anticonv', 'varum', 'stump', 'abnormalt', 'amer', 'delivery', 'impct', 'imperfecta', 'hematological', 'scuba', 'retroperiton', 'enlargement', 'conduc', 'varn', 'juvenile', 'toe', 'kuru', 'inversion', 'following', 'hematocrit', 'cyc', 'strictur', 'cervical', 'imp', 'angio', 'substanc', 'thmb', 'uninodular', 'obstructive', 'brst', 'band', 'diastolic', 'nasopharyx', 'hprosmlr', 'head', 'striae', 'schistosomiasis', 'resonance', 'rflux', 'col', 'solvent', 'brth', 'pncllins', 'hiv', 'tabes', 'optica', 'delusional', 'consl', 'ski', 'prophyl', 'alcohol', 'jugular', 'respiration', 'mycotic', 'necrosis', 'diastasis', 'sedate', 'midcarpal', 'vara', 'intestin', 'colitis', 'accommodation', 'multiple', 'ectpc', 'exercse', 'excipien', 'precip', 'parenchyma', 'monoamine', 'systemc', 'hydcarb', 'perp', 'chf', 'dilation', 'mcyc', 'sub', 'pad', 'mther', 'reduced', 'electro', 'trichomonal', 'urt', 'prolonged', 'cesarean', 'simplex', 'cheyne', 'acc', 'gastroduodenal', 'accidental', 'tight', 'incidental', 'sleep', 'deep', 'epimembranous', 'peritoneal', 'hemodialysis', 'rectovag', 'bancroftian', 'napkin', 'deviation', 'cubitus', 'polydactyly', 'risk', 'rec', 'lightning', 'lymphprof', 'stom', 'histio', 'pneumoniae', 'lack', 'nitidus', 'subendo', 'humerus', 'urination', 'long', 'argyll', 'phenthiaz', 'ankylosis', 'dtp', 'materials', 'pneumothorx', 'hy', 'muscl', 'miotics', 'hypertelorism', 'branchial', 'nodos', 'transvestic', 'bronchog', 'psychostim', 'ect', 'antineoplastic', 'squam', 'prep', 'ovary', 'pesticides', 'suberosis', 'global', 'coronavirus', 'nephrotic', 'conservatn', 'tetanus', 'nigricans', 'enthesopathy', 'parapertussis', 'disablty', 'barotrauma', 'blepharospasm', 'cephal', 'mycobacteria', 'tularemia', 'hungry', 'glottis', 'restrain', 'osteochondrosis', 'pharyngoconjunct', 'membranoprolif', 'excs', 'psymotr', 'arousals', 'intracranial', 'sarcosporidiosis', 'restless', 'ridge', 'noninflam', 'luts', 'health', 'effusion', 'sebrrheic', 'dura', 'lagophthalm', 'k', 'abn', 'hemiplegia', 'bucket', 'fascia', 'vision', 'cranl', 'marginal', 'premises', 'aware', 'vap', 'defcnt', 'adm', 'canoe', 'cortx', 'platfrm', 'chest', 'concussion', 'parvovirus', 'antiphospholipid', 'routine', 'digestants', 'cardiogenic', 'distance', 'brf', 'ganong', 'elsewhr', 'removal', 'scleral', 'aspergillosis', 'subsequent', 'cystica', 'altered', 'occlsn', 'ultrasnd', 'ketoacd', 'hematol', 'needle', 'disordr', 'field', 'electromyogram', 'keratoconjunct', 'wtnos', 'hema', 'antepa', 'pyemia', 'ami', 'dermatomycoses', 'threat', 'd', 'imcompl', 'metabl', 'salicylates', 'lesion', 'grf', 'neuropthy', 'postsurg', 'chondroectoderm', 'mucoid', 'hepatomegaly', 'lymph', 'lacrimal', 'chilblains', 'dep', 'function', 'skl', 'primigravid', 'follicular', 'hyperventilation', 'privat', 'arm', 'dyspareunia', 'aflatox', 'enteritis', 'fossa', 'nitrogen', 'antibiotics', 'radioulnar', 'vulnif', 'andepres', 'vess', 'puckering', 'fb', 'chordae', 'cavity', 'orthoptic', 'peritoneum', 'horn', 'microangiopathy', 'affectiv', 'visible', 'brow', 'deployment', 'q', 'complic', 'otitis', 'protect', 'cardiovasc', 'vulva', 'postconcussion', 'cur', 'tuberous', 'myiasis', 'keratoconus', 'nonwork', 'desensitiza', 'bronchospasm', 'natural', 'suppr', 'marching', 'mcrg', 'creutzfeldt', 'hypotensive', 'traction', 'urn', 'displace', 'deform', 'form', 'effluent', 'musculoskel', 'ventral', 'slowing', 'metacarp', 'hidradenitis', 'granul', 'hypervitaminosis', 'compres', 'unkn', 'erysipelothrix', 'sphenoid', 'erythema', 'fistula', 'sup', 'hypotension', 'hemorrhoid', 'selective', 'ntv', 'relative', 'ur', 'nasopharyngitis', 'infectiosum', 'hyperostosis', 'intravasc', 'fold', 'erlk', 'spirochetal', 'nervous', 'intestines', 'complicting', 'outcome', 'borderline', 'spondylosis', 'abd', 'biliuria', 'hiccough', 'dosag', 'like', 'ariboflavinosis', 'baseball', 'ecmo', 'urethrts', 'prolaps', 'anterolateral', 'organphos', 'artific', 'pectoris', 'hematocele', 'influenzae', 'carbuncle', 'catarct', 'adolescence', 'therapy', 'monoclonal', 'occulta', 'court', 'california', 'von', 'spontan', 'ischem', 'infusion', 'small', 'nonorganic', 'conn', 'saluretics', 'screen', 'hirudiniasis', 'referred', 'mandibular', 'traumatc', 'nerv', 'contusions', 'strab', 'syndactyl', 'algae', 'vulvr', 'subscapularis', 'stromal', 'bullous', 'infer', 'glucse', 'myelo', 'orchitis', 'lips', 'bitot', 'scapul', 'vasodilat', 'combin', 'arthralgia', 'curnt', 'tol', 'scabies', 'after', 'avian', 'malunion', 'ortho', 'goodpasture', 'prosth', 'spina', 'rmvl', 'malt', 'lsn', 'cl', 'externl', 'dagger', 'metatarsophalang', 'patent', 'infilt', 'cabl', 'stream', 'schizoaff', 'w', 'sepsis', 'windsrf', 'stair', 'sediment', 'attrition', 'tibialis', 'bcg', 'suicidal', 'capsulat', 'depend', 'metal', 'perforation', 'treatmnt', 'senile', 'nonsurg', 'rupt', 'brnchlts', 'herpesvir', 'hydatidiform', 'impotence', 'symphy', 'tracheitis', 'asymmetry', 'pedest', 'aftercare', 'hypertroph', 'uter', 'hypr', 'deficiencies', 'quadrplg', 'gastro', 'bowel', 'tidalwave', 'pellets', 'clstrdium', 'manifestation', 'hyprtrophic', 'excip', 'emptying', 'elong', 'left', 'hair', 'postphlbtc', 'machinery', 'neutropenia', 'vep', 'oxazolidin', 'anatom', 'visual', 'sexualism', 'sanguin', 'asp', 'steam', 'keratitis', 'chondrodystrophy', 'epidemic', 'postoperative', 'herpetiformis', 'rst', 'pelvic', 'impetigo', 'median', 'hrt', 'intoleran', 'mooren', 'membranous', 'should', 'infectious', 'alastrim', 'xeros', 'laxity', 'frts', 'adrenogenital', 'nutrit', 'chorioretinit', 'tachypnea', 'panuveitis', 'sin', 'strptococcus', 'granuloma', 'congn', 'missile', 'polyphagia', 'organiz', 'enlargmnt', 'int', 'tpa', 'aggressiv', 'cells', 'pernicious', 'musclr', 'crcnoid', 'chordee', 'hypopigmentation', 'olfactory', 'demulc', 'negatv', 'uncon', 'parasympatholytc', 'pesticid', 'dosage', 'hmophlus', 'orbit', 'disinfectants', 'toxocariasis', 'amputation', 'aftrce', 'del', 'vag', 'atyp', 'trachea', 'agents', 'nutritional', 'neurpthy', 'neonatorum', 'hyprt', 'disproportn', 'decr', 'retromolar', 'perfringens', 'multiplex', 'gall', 'enophthalmos', 'postendoscopy', 'inversus', 'penicillins', 'leukoplakia', 'syncope', 'eye', 'impaired', 'trapezium', 'scrotum', 'unconfirm', 'neuritis', 'dtap', 'zoster', 'strychnine', 'extremity', 'high', 'intrauter', 'microcystoid', 'mover', 'antilipemics', 'grilling', 'pelvis', 'prolifer', 'contracept', 'hyperaldosteronism', 'suppurativ', 'fluoride', 'unspc', 'leakage', 'fragilis', 'herid', 'sclero', 'baby', 'unspecifd', 'suscept', 'maxillary', 'concuss', 'meningococc', 'privation', 'hypercementosis', 'arms', 'prematurity', 'aldsternsm', 'jaund', 'parotid', 'hemicrania', 'sinusitis', 'encephalocele', 'utl', 'versus', 'thromb', 'planitis', 'monoc', 'whooping', 'dysplas', 'protan', 'acromegaly', 'rail', 'separate', 'fanciers', 'exps', 'thrmbs', 'intoxic', 'desig', 'inferior', 'residual', 'autosomal', 'nystagm', 'bathtub', 'anisakiasis', 'carcinoma', 'resorption', 'lepromatous', 'vaginitis', 'ring', 'stpfth', 'noninfec', 'dermatochalasis', 'hematometra', 'raynaud', 'cmp', 'agoraphobia', 'ntc', 'descrp', 'laryngotracheitis', 'famil', 'measle', 'ulceratve', 'tis', 'convergenc', 'schizoid', 'parapharyngeal', 'duodenum', 'supvsn', 'achv', 'fibroplasia', 'mastocytosis', 'htr', 'cross', 'deaf', 'mangmt', 'anisocoria', 'strength', 'av', 'curv', 'olecran', 'labyrinth', 'equatorial', 'accommodative', 'mul', 'infant', 'paraplegia', 'coagulat', 'carnitine', 'hos', 'gangrene', 'cathartics', 'asbestosis', 'kayser', 'orgnsm', 'kyasanur', 'neonate', 'prophylactic', 'vera', 'voyeurism', 'pneumonia', 'isthmus', 'achilles', 'lacrimogenic', 'atten', 'dom', 'loop', 'solv', 'deterg', 'antidiarrhea', 'lymphedema', 'nonrupt', 'isch', 'anmaly', 'tendon', 'than', 'perfus', 'cliff', 'personality', 'evid', 'bn', 'myeloma', 'sensr', 'leukomalacia', 'madura', 'rope', 'desquamatv', 'pta', 'toxoplasmos', 'isopropyl', 'hypothyroidism', 'bark', 'gstrontest', 'alt', 'consanguinity', 'interven', 'iridocyl', 'prime', 'do', 'hospitl', 'anencephalus', 'decision', 'acu', 'uplmb', 'aspirator', 'impairemt', 'cataplexy', 'cv', 'sacroiliitis', 'overlap', 'umbil', 'tracheostomy', 'adj', 'anaerobic', 'exud', 'fl', 'before', 'antidotes', 'vocal', 'isoimmun', 'prg', 'uti', 'hyperphoria', 'cardiotonics', 'bagassosis', 'hypoglycemia', 'board', 'sprain', 'concept', 'sedatives', 'duboisii', 'metrorrhagia', 'convergence', 'trch', 'ix', 'trypanosoma', 'objective', 'septicemic', 'climact', 'boarding', 'run', 'bleb', 'implantation', 'perfusn', 'uveitis', 'leukemoid', 'viii', 'exstrophy', 'indir', 'abdomin', 'osteomyelitis', 'arthropathy', 'varic', 'arom', 'vessel', 'sbstn', 'bps', 'paragonimiasis', 'atticoantral', 'milestones', 'nonspcf', 'ossicl', 'urnr', 'histidine', 'eustachian', 'prosthtc', 'sensor', 'teething', 'brd', 'three', 'src', 'frm', 'retrovirus', 'phos', 'forearm', 'observe', 'posticum', 'pust', 'hypopotassemia', 'dorsal', 'barbiturates', 'toxoplasmosis', 'knitting', 'retinal', 'ascending', 'road', 'ophthalmia', 'iron', 'retinoschisis', 'mgrn', 'spines', 'separat', 'subjective', 'dark', 'colstmy', 'psg', 'phenomenon', 'emb', 'uncomp', 'hypercarotinemia', 'diseases', 'broad', 'malaise', 'complcation', 'inguinal', 'endometritis', 'juven', 'course', 'straining', 'filamentary', 'shellfish', 'colles', 'gstr', 'orth', 'extravasn', 'sector', 'septum', 'psychomot', 'incr', 'hymen', 'orbital', 'deltoid', 'conjunctival', 'mgkrycyt', 'convn', 'subclavian', 'anaerobes', 'mydriasis', 'cnsl', 'mycs', 'endomyocardial', 'sedation', 'dorsalis', 'patellae', 'neovascular', 'length', 'gaze', 'tendn', 'excretion', 'hypersomnia', 'hemat', 'corros', 'hamartoses', 'effusn', 'cerebri', 'dam', 'rib', 'prphylct', 'spirillary', 'shiga', 'antepartum', 'shoulder', 'trct', 'cel', 'mittelschmerz', 'septc', 'gestation', 'fit', 'glutethimide', 'fecal', 'induce', 'puerperal', 'rehabilitation', 'tor', 'dissection', 'induced', 'loose', 'fluency', 'hallux', 'meningococcemia', 'elective', 'leukemia', 'endodontic', 'thrombophlebitis', 'anoth', 'myelofibrosis', 'nck', 'tubrclma', 'quadricep', 'mth', 'nonautlg', 'suprfcl', 'sprfc', 'interphalangeal', 'athletes', 'close', 'intn', 'stud', 'nonsenile', 'postpar', 'mid', 'taeniasis', 'ulcertve', 'mamm', 'johnson', 'panic', 'sbrheic', 'menopausal', 'frostbite', 'thrombocytopen', 'uterus', 'osteoarthrop', 'scolios', 'exophthalmos', 'mercurial', 'cnstrt', 'instrumnt', 'periapical', 'distal', 'equ', 'brachial', 'oxidase', 'intrst', 'aut', 'ligation', 'lip', 'chromium', 'emollient', 'wegener', 'toxoplasma', 'reflex', 'activity', 'malfor', 'sit', 'snowboard', 'hazrd', 'dystrophy', 'lymphatic', 'screening', 'hemolytic', 'hypogammaglobulinem', 'exotrop', 'ddnts', 'powered', 'bctr', 'fasciolopsiasis', 'drusen', 'salts', 'spinal', 'autm', 'serlgc', 'middle', 'disor', 'influenza', 'adenovirus', 'hypoxia', 'cerebrospinal', 'anticonvulsant', 'epicondylitis', 'candida', 'papilledema', 'event', 'caus', 'depersonalization', 'writers', 'hemochromatosis', 'felty', 'ocl', 'erup', 'abortus', 'anterior', 'divorce', 'distortions', 'supraspinatus', 'fracture', 'hyperalimentation', 'workers', 'primus', 'host', 'extr', 'neuromyelitis', 'adhesns', 'osteomyelit', 'goiter', 'polyglandul', 'implant', 'occlus', 'pku', 'episodic', 'sadism', 'infectv', 'nontraum', 'flatul', 'dysplasia', 'fungi', 'acetabulum', 'mandib', 'prolapse', 'inconclusive', 'leptospiral', 'inflamm', 'myositis', 'delayed', 'intrinsc', 'hypotens', 'cognition', 'alopecia', 'omphalitis', 'cong', 'plan', 'osteochondros', 'disprop', 'obliq', 'seminal', 'wasting', 'dv', 'employee', 'arcuate', 'isoim', 'nasopharyn', 'syncytial', 'parametrium', 'articular', 'inlet', 'short', 'clostrid', 'pemph', 'ectropion', 'gravis', 'rhym', 'abdominal', 'seed', 'encysted', 'hoffmann', 'derail', 'damg', 'meningococcal', 'cycl', 'struc', 'pl', 'susc', 'cycle', 'inadequate', 'geniculate', 'magnesium', 'duodenal', 'drowning', 'moyamoya', 'loosening', 'mult', 'hard', 'alvlar', 'ectasia', 'anemias', 'ac', 'dl', 'compress', 'alkali', 'chondrodystrphy', 'explosion', 'serum', 'acoustic', 'refractive', 'aortocoronary', 'xi', 'crdc', 'wrong', 'incon', 'neuropacemaker', 'epiphyseal', 'date', 'hypoparathyroidism', 'biceps', 'retinoshisis', 'ndlr', 'inc', 'fireworks', 'retention', 'benign', 'vestibulr', 'nuck', 'states', 'persistent', 'gu', 'issue', 'leptospirosis', 'ch', 'cholerae', 'grav', 'autmtc', 'polyneuropathy', 'injury', 'sick', 'menopause', 'down', 'letter', 'introd', 'plac', 'pediculosis', 'relocation', 'aged', 'obscur', 'skin', 'digestve', 'metalworking', 'intest', 'sweating', 'hemangioma', 'reverse', 'corpus', 'vrus', 'cochlea', 'malar', 'gngrene', 'lymphangioleiomyomatosis', 'sterilization', 'sigmoid', 'thrm', 'self', 'abnorm', 'evdnc', 'blepharoconjunct', 'pruritus', 'adhesion', 'complaint', 'monochor', 'infil', 'refract', 'petroleum', 'fingr', 'cyanotic', 'factor', 'diaphragm', 'agenesis', 'labyrn', 'agorphobia', 'use', 'lowlmb', 'mammary', 'bengn', 'thirst', 'chlorine', 'heroin', 'frequency', 'undeter', 'derm', 'extreme', 'molluscum', 'earth', 'remote', 'trigonitis', 'inorg', 'cartil', 'opium', 'systolic', 'sirs', 'transv', 'at', 'echinococcosis', 'dysgenesis', 'mitr', 'senso', 'wtrcrft', 'effects', 'aorta', 'bursa', 'lobe', 'aldosternsm', 'evidence', 'yaws', 'trma', 'antisoc', 'emergncy', 'arthropath', 'pyemic', 'inflm', 'welander', 'sensry', 'dissecans', 'leuk', 'unequal', 'resusctate', 'chutist', 'fibrosis', 'pregnancy', 'metacarpophaln', 'watrcrft', 'postproc', 'unc', 'pseudotumor', 'ulceroglandul', 'rappelling', 'handheld', 'bld', 'vasoplasty', 'unsat', 'arter', 'duct', 'histoplasma', 'sonnei', 'dressing', 'clavicl', 'chyluria', 'metab', 'pept', 'clothing', 'microlith', 'polyarthritis', 'labrm', 'surg', 'sympth', 'nonadmin', 'schiz', 'collis', 'diastol', 'graft', 'spectator', 'training', 'diphtheritic', 'ostium', 'liver', 'vesiculitis', 'allrgy', 'hypertrophy', 'premenstrual', 'dist', 'flexneri', 'vaccn', 'schizophren', 'deriv', 'hypercholesterolem', 'delta', 'dermatit', 'inspiss', 'fail', 'anim', 'cysts', 'transverse', 'hit', 'volcanic', 'trichomoniasis', 'grd', 'hypotropia', 'congen', 'kidneys', 'mononeur', 'repro', 'orgn', 'slipping', 'musculoskletl', 'renovasc', 'symptoms', 'ophthalmopleg', 'cndct', 'heterophyiasis', 'necro', 'cage', 'nile', 'excitation', 'ethyl', 'thumb', 'obstacle', 'aneurym', 'malab', 'ig', 'rhomboid', 'crocheting', 'anter', 'subluxation', 'radioac', 'dysuria', 'genet', 'negative', 'west', 'pilar', 'villonod', 'amput', 'narcissistic', 'healing', 'contagious', 'pediculus', 'subch', 'pillars', 'lumbar', 'debility', 'pressure', 'npl', 'teeth', 'apraxia', 'cephalic', 'patella', 'applianc', 'cntct', 'medu', 'interrupt', 'prog', 'excitativ', 'glenoid', 'eat', 'ut', 'dub', 'relax', 'vitelliform', 'polypoid', 'postop', 'sled', 'tamponade', 'unpower', 'radial', 'epigastric', 'atres', 'alternating', 'langerhans', 'osteoart', 'fatigue', 'cat', 'scratch', 'oxide', 'house', 'pituit', 'peptic', 'osse', 'otogenic', 'of', 'intrathoracic', 'plantar', 'retinoph', 'vernal', 'veh', 'grn', 'capitate', 'aggress', 'pneumo', 'typhoid', 'sbdrm', 'actnc', 'conditions', 'cmplc', 'perineum', 'pubis', 'bartter', 'peritonitis', 'sacrum', 'hyperplasia', 'inferolateral', 'spot', 'fistul', 'parts', 'elev', 'el', 'basl', 'fundus', 'transpos', 'arthr', 'triplet', 'cystic', 'russia', 'fugue', 'deter', 'myelophthisis', 'special', 'midtarsal', 'omphalocele', 'hyprplas', 'drg', 'tonsil', 'highly', 'bipol', 'organic', 'exfl', 'hmplg', 'distrs', 'cholelith', 'micro', 'mucocutan', 'blepharoconjunctivit', 'us', 'constipation', 'asslt', 'posture', 'interphalang', 'injry', 'intrahepat', 'cardiomy', 'traffic', 'yaba', 'kaschin', 'cerebellar', 'antep', 'hmiplg', 'myelopat', 'missed', 'bifida', 'cult', 'mater', 'treatd', 'rapidly', 'circuit', 'p', 'htlv', 'stokes', 'keratoconjunctiv', 'strabismic', 'vitreoretinal', 'less', 'depressive', 'cooking', 'sickle', 'rup', 'sympath', 'web', 'intracerebral', 'pineal', 'who', 'postmenopausal', 'response', 'urth', 'hepat', 'exer', 'poliomyelitis', 'dev', 'country', 'deliv', 'pulmon', 'neglect', 'oculoglandular', 'intermittnt', 'lev', 'exter', 'hip', 'upr', 'reattached', 'vertebr', 'tail', 'metagonimiasis', 'around', 'smear', 'bacteroides', 'fng', 'tie', 'personal', 'nox', 'tib', 'burn', 'erythematos', 'thrombocytpenia', 'coccidioid', 'tanapox', 'cuneiform', 'trachoma', 'gi', 'effluvium', 'pt', 'hypoactive', 'bldg', 'pancytopna', 'neuro', 'analgesc', 'lwr', 'position', 'ethmoidal', 'complx', 'coracohumeral', 'toxoid', 'observ', 'vesiculit', 'antineo', 'dye', 'insufficnt', 'leishmanias', 'prs', 'pediatric', 'myopathy', 'hf', 'prfound', 'necrolysis', 'erythma', 'retrovert', 'amniotic', 'sprue', 'nos', 'gout', 'actv', 'exam', 'lipidosis', 'residential', 'solid', 'pasngr', 'calcaneofibular', 'progess', 'osseo', 'radiocarpal', 'stature', 'egg', 'ingrowing', 'sterile', 'derivat', 'epil', 'membrane', 'racquet', 'paraphrenia', 'radiogrphc', 'polishes', 'periodont', 'necrotizing', 'tot', 'analgsc', 'folate', 'counsl', 'hazard', 'balanoposthitis', 'analgesics', 'intrathor', 'rmsion', 'vacuum', 'nodosum', 'choroid', 'uterine', 'card', 'complicated', 'hyperfunction', 'candidias', 'mgrne', 'proliferat', 'diamni', 'rout', 'syphil', 'skate', 'pick', 'aortitis', 'iatrogenic', 'constant', 'compnds', 'typing', 'arrested', 'varicos', 'colstomy', 'coagulatn', 'groom', 'seven', 'inclusion', 'cornea', 'condctv', 'postinjection', 'emoll', 'palm', 'antipyr', 's', 'polyarteritis', 'postabort', 'galactocele', 'hyperkinetic', 'giardiasis', 'subcorneal', 'an', 'iatrogen', 'weights', 'illness', 'hallucinog', 'hymenal', 'schnk', 'hymenolepiasis', 'ascvd', 'dilut', 'abv', 'beryllium', 'opioid', 'part', 'prnt', 'cowpox', 'forcep', 'lumbosac', 'nonpetrol', 'dipetalonemiasis', 'drmatitis', 'fissural', 'oropharyn', 'wake', 'breathing', 'memory', 'aortc', 'access', 'mye', 'lng', 'pylorus', 'jaw', 'fragmnt', 'recurrnt', 'psychostimulants', 'congest', 'plv', 'deficiency', 'diplegia', 'dsord', 'arthropod', 'aspirin', 'cryptogenic', 'aerobacter', 'deafness', 'able', 'natrl', 'mat', 'fibrous', 'interac', 'herpesvirus', 'paraldehyde', 'movemt', 'size', 'cerebrovascular', 'moving', 'malayan', 'eosinophilic', 'family', 'protrusio', 'wmd', 'epi', 'prost', 'crash', 'glnd', 'hemiplga', 'ag', 'bereavement', 'vscl', 'yatapoxvirus', 'malfrm', 'man', 'reptv', 'ureteral', 'excitement', 'panhypopituitarism', 'shock', 'topic', 'tongue', 'circumscribe', 'erythmtsquamous', 'fmr', 'trial', 'communicate', 'ulnohumeral', 'trichinosis', 'chlorl', 'child', 'submersion', 'placentl', 'disloca', 'saddle', 'prostatocystitis', 'counseling', 'gentc', 'axis', 'sb', 'advc', 'sec', 'lipidoses', 'dvrtclo', 'rstn', 'lowr', 'queensland', 'due', 'paralytic', 'inter', 'tropical', 'polymyositis', 'hearng', 'pes', 'calcium', 'nevus', 'stomatitis', 'mucosits', 'postcoital', 'biologicals', 'ischemia', 'difus', 'unaggr', 'flr', 'bdy', 'phenothiaz', 'remis', 'eosinophilc', 'polyneuropath', 'inoc', 'treat', 'pol', 'sedtv', 'crit', 'suspc', 'cil', 'inv', 'thorax', 'acquired', 'mismatch', 'flexor', 'anthrax', 'toxic', 'rotation', 'mapl', 'routin', 'driver', 'endocrine', 'street', 'arboviral', 'microcalcif', 'bipolar', 'caffeine', 'depress', 'pollen', 'capsular', 'effec', 'hyperacusis', 'proteinuria', 'militry', 'inactive', 'cervicalgia', 'heart', 'eclampsia', 'hepatorenal', 'telangiectasia', 'hypertrph', 'vitreoretina', 'extraocul', 'exophoria', 'enterost', 'splitting', 'ii', 'nephrit', 'latent', 'inflam', 'duod', 'confinement', 'obs', 'gest', 'foundling', 'diab', 'passiv', 'nonvenereal', 'appy', 'layer', 'malpresen', 'bnign', 'procedure', 'i', 'polyarthrop', 'weapons', 'libido', 'electric', 'cytolgy', 'ascend', 'adhes', 'fish', 'disrdr', 'shower', 'staphylococc', 'pubocerv', 'gastroenterit', 'clinic', 'sect', 'supervis', 'brucella', 'crystal', 'occupationl', 'recep', 'dentofacial', 'tuberosity', 'kd', 'arch', 'monoarthritis', 'haz', 'cold', 'perforated', 'synostosis', 'islet', 'retain', 'dent', 'photokeratitis', 'crft', 'wait', 'immuno', 'externum', 'defined', 'tympnsclr', 'recession', 'cmpnd', 'eyeball', 'pisiform', 'posttransfusion', 'sol', 'bslr', 'gnathostomiasis', 'herpesvirs', 'misdirection', 'hydrocephalus', 'lgsil', 'iodine', 'dystr', 'disfigurements', 'iniencephaly', 'splenic', 'embolsm', 'tndr', 'coccidioidomycosis', 'expose', 'hypertensive', 'nose', 'encephalopathy', 'contct', 'fetopel', 'conjunct', 'imnprlf', 'vertbrobaslr', 'postinfectious', 'pupil', 'fumigants', 'or', 'mono', 'place', 'cyclotropia', 'madelung', 'malocclusion', 'ganglionitis', 'amyotrophic', 'crushing', 'pnumn', 'volunteer', 'sound', 'sequelae', 'fight', 'alzheimer', 'myelofi', 'testes', 'lipoprotein', 'domestc', 'hirsutism', 'balanitis', 'friedreich', 'subarach', 'intraven', 'antrum', 'impaction', 'erythrm', 'scrn', 'subtrochanteric', 'ext', 'necr', 'endometriosis', 'vesicant', 'machine', 'osteochondropathy', 'dermat', 'passenger', 'liv', 'tympan', 'hypofunc', 'pseudocowpox', 'petit', 'contrctr', 'vestibule', 'disturbances', 'related', 'weapn', 'primigrav', 'liquid', 'melanoma', 'allergy', 'exophthalm', 'epididymit', 'glossopharyng', 'hypotnsion', 'postherpes', 'insensitvty', 'anophthalmos', 'transitory', 'varicosity', 'scleros', 'cva', 'contaminated', 'vocational', 'antimalarials', 'postperfusion', 'triquetral', 'by', 'nuclear', 'vulval', 'dieulafoy', 'malfunc', 'thracabd', 'claudct', 'modified', 'postlaminect', 'meconium', 'stimulant', 'vertebrae', 'norm', 'watercrft', 'ot', 'dislocat', 'hunting', 'junction', 'estrangmemt', 'nonrenal', 'reac', 'malaria', 'fibroadenosis', 'calcificat', 'dysenteriae', 'ideation', 'weeks', 'postgastric', 'syphilis', 'syringomyelia', 'tetany', 'inhibited', 'straig', 'snake', 'electrocution', 'expectorants', 'surveil', 'cholelh', 'pathol', 'transf', 'problem', 'circum', 'intercostal', 'scorpion', 'agnt', 'subcu', 'anom', 'bne', 'wtrcraft', 'avoidant', 'coxsackie', 'cort', 'intrcsp', 'uncntrld', 'material', 'epidem', 'adenoid', 'explantatn', 'keratoderma', 'exm', 'mycetomas', 'helminthiasis', 'heel', 'conjugat', 'medulloadrenal', 'fires', 'cmplcatn', 'two', 'polyp', 'opacities', 'postmastoid', 'postsurgery', 'efct', 'amyotrophy', 'duane', 'smooth', 'hodg', 'pubertal', 'indwell', 'artif', 'mature', 'defecation', 'antidepressnt', 'cholera', 'bronc', 'trench', 'pigmentary', 'uncompl', 'opisthorchiasis', 'loiasis', 'antenatal', 'round', 'intussusception', 'lawn', 'digeorge', 'essntial', 'patient', 'carcnoid', 'osseous', 'lipotropic', 'alc', 'wbc', 'military', 'sting', 'lymphocytopenia', 'pap', 'paratyph', 'lap', 'psychosocial', 'embolism', 'nonsup', 'gang', 'control', 'irritant', 'fum', 'pinta', 'louping', 'resist', 'reduct', 'endemic', 'prostatic', 'skew', 'occiptpost', 'histolog', 'epicranial', 'macroglobulinemia', 'soemmering', 'hemosiderosis', 'chron', 'belly', 'sedat', 'blasting', 'administrtve', 'ossicles', 'thoracic', 'ges', 'beta', 'inflammation', 'asprglosis', 'rheumat', 'otalgia', 'iritis', 'tarsal', 'areata', 'dif', 'arthritis', 'pulmonary', 'orign', 'pass', 'attem', 'herpangina', 'restore', 'superfic', 'monoxide', 'isol', 'patellar', 'suis', 'myasthenia', 'mononeuritis', 'rt', 'decrease', 'tranquilzr', 'shortening', 'plana', 'hemorrhoids', 'reactn', 'erythematosus', 'pulsating', 'expres', 'cystostomy', 'shaken', 'educational', 'ar', 'tibiofibul', 'bite', 'pyogenic', 'arthroscopc', 'propionic', 'opacity', 'hypotensn', 'hemolysis', 'glasses', 'tunnel', 'keratosis', 'more', 'benzodiaz', 'vasectomy', 'erythem', 'absc', 'nonspecif', 'tension', 'dial', 'neurosyph', 'vincent', 'flushing', 'sptc', 'chr', 'medull', 'antidepressants', 'kn', 'athrscl', 'parinfluenza', 'scope', 'urban', 'anti', 'acrocephalosyndactyly', 'restoration', 'during', 'aud', 'phlebitis', 'subacute', 'entering', 'eyes', 'pure', 'defibrination', 'gastrin', 'mem', 'externa', 'dystrphy', 'individ', 'bk', 'sbc', 'placen', 'prenatal', 'lat', 'uncontr', 'intraut', 'uncomplicated', 'menstruation', 'pinna', 'repeat', 'compression', 'gastrojejun', 'acne', 'entrtoxgn', 'opt', 'nystagms', 'exercise', 'americanus', 'neutrophils', 'iga', 'pouch', 'from', 'chlmyd', 'disc', 'sphnctr', 'precocity', 'trach', 'endocardit', 'posterior', 'internucl', 'spont', 'rotatr', 'alone', 'positive', 'retrn', 'giddiness', 'appl', 'foc', 'crcnd', 'sunstroke', 'trnsplnt', 'occupatnal', 'tibial', 'canis', 'myelopathies', 'rubella', 'bicipital', 'prophylac', 'oc', 'delus', 'disorder', 'paraurethral', 'allerg', 'noncompliance', 'urgency', 'rhinorrhea', 'limbs', 'constption', 'psychotrop', 'cervcl', 'eruptions', 'sequestration', 'ante', 'utervaginal', 'postsurgical', 'dm', 'venomous', 'malign', 'complc', 'syn', 'latex', 'pestic', 'ectp', 'distress', 'sports', 'nephritis', 'umbilical', 'klippel', 'glycogenosis', 'neonatal', 'anomalies', 'plnd', 'marker', 'thyrocalciton', 'broken', 'myotonia', 'relapse', 'hnd', 'lymphosarcoma', 'systm', 'headache', 'ophth', 'cicatricial', 'sternoclavicular', 'large', 'distl', 'antirheumatic', 'hormone', 'pul', 'thrombophleb', 'cleft', 'jetlag', 'iridis', 'procd', 'anem', 'supplement', 'fissure', 'melanom', 'bord', 'musculskel', 'sphenoidal', 'intern', 'schilder', 'wood', 'postphlebtc', 'cholesterolosis', 'skier', 'babesiosis', 'premenopause', 'taenia', 'uncomplicat', 'shldr', 'laryngopharyngitis', 'presenile', 'criminal', 'tranquilizer', 'disord', 'lumb', 'granular', 'gangrenosum', 'circumcision', 'mellitus', 'trnsfrm', 'salivary', 'lambert', 'aircraf', 'amebic', 'japonicum', 'up', 'sars', 'cryptococcal', 'par', 'condyl', 'shigella', 'amebae', 'org', 'degneration', 'quadriplegia', 'malnutrition', 'rumination', 'hepatitis', 'papillomata', 'malabsorption', 'tobagn', 'melitensis', 'macrotia', 'insect', 'hb', 'weaning', 'europe', 'find', 'hypomagnes', 'labia', 'prev', 'postmeno', 'intraepth', 'intrathorac', 'personl', 'anthelmintics', 'hemothorax', 'ethiop', 'neuroendo', 'machin', 'allescheriosis', 'right', 'xerotica', 'trunk', 'periodontal', 'varnt', 'jct', 'op', 'viral', 'dermatoglyphic', 'anomal', 'post', 'seborrhea', 'psy', 'complicat', 'pr', 'coma', 'severe', 'subperi', 'cutan', 'postinflam', 'diff', 'made', 'playing', 'nocturnal', 'antidiarrh', 'slv', 'sqam', 'staphyloma', 'enema', 'cervicocranial', 'cav', 'proced', 'adrenal', 'catheteriz', 'aesthetics', 'impulse', 'std', 'benzodiazepine', 'hypopyon', 'lacr', 'branch', 'lichenification', 'traumtc', 'ossification', 'forehead', 'pseudoexfol', 'chloramphenicol', 'mplnt', 'hemianopsia', 'hypersplenism', 'chin', 'multifoc', 'contus', 'metatarsals', 'penetration', 'anoxic', 'chromoso', 'complex', 'hgsil', 'pow', 'vacc', 'amniocent', 'glaucomatous', 'volkmann', 'omsk', 'storm', 'ntr', 'aort', 'mediastinitis', 'aglossia', 'hemor', 'tubotympan', 'convales', 'pellagra', 'atrophy', 'dengue', 'catar', 'convulsns', 'school', 'precipitate', 'chamb', 'glossophar', 'bilious', 'multigravd', 'dmg', 'hang', 'battering', 'light', 'cava', 'cyclic', 'reflux', 'claud', 'parahaem', 'macrolides', 'labr', 'bowing', 'paravaccinia', 'attempted', 'suprcondyl', 'aniseikonia', 'sickness', 'trachomatis', 'astragalus', 'initial', 'diord', 'salpingo', 'drug', 'hprsm', 'heterosexual', 'beard', 'confusion', 'inertia', 'bleed', 'age', 'rigidus', 'anorexia', 'dlta', 'blepharophimosis', 'deprivatn', 'rhinitis', 'nipple', 'malformatn', 'gambling', 'occl', 'aneurysmal', 'synth', 'injection', 'pseudopolyposis', 'push', 'silica', 'sunburn', 'scar', 'vasculitis', 'phalanges', 'empl', 'ramus', 'insuffic', 'alpha', 'premature', 'implnt', 'peroneal', 'coron', 'dietary', 'spur', 'prematr', 'corns', 'pln', 'aphasia', 'multiforme', 'cal', 'arthropods', 'in', 'traumatic', 'urinar', 'vscurt', 'biliary', 'industr', 'pyelonephrit', 'reconst', 'sev', 'eryth', 'perforans', 'groin', 'tia', 'steatorrhea', 'hyprhidros', 'thrd', 'shortness', 'survey', 'primigravida', 'ldh', 'volume', 'thrombosis', 'nonpyogen', 'shlder', 'biolog', 'amyloidosis', 'generalizd', 'dyssnrgia', 'macrocheilia', 'scooter', 'ova', 'dur', 'complication', 'rhinovirus', 'string', 'consult', 'dysfunct', 'pingueculitis', 'pancreatitis', 'asc', 'catatonia', 'calcif', 'hypoglossal', 'asy', 'vac', 'osteogenesis', 'multiparity', 'benzene', 'medical', 'neurofibromatosis', 'tags', 'unilat', 'senility', 'path', 'util', 'spin', 'yoga', 'utero', 'bal', 'esophoria', 'bronchopneumonia', 'miotic', 'hemopericardium', 'bartholin', 'adoles', 'monoplegia', 'instability', 'singl', 'hypothyroidsm', 'skelet', 'products', 'mastitis', 'nuc', 'underimmunizn', 'mastoid', 'palat', 'emol', 'imno', 'trichilemmal', 'lst', 'volleyball', 'sounds', 'epith', 'cruciat', 'memb', 'postabortion', 'prosthesis', 'piedra', 'mental', 'b', 'crew', 'trypanosomiasis', 'cardiospasm', 'shunt', 'heteronymous', 'causalgia', 'sword', 'exocervix', 'apical', 'rectosigmoid', 'cnd', 'derangmnt', 'simpl', 'urge', 'duoden', 'amnestic', 'remove', 'glossitis', 'impr', 'gastroenteritis', 'tritan', 'microcephalus', 'sm', 'exhaust', 'tympanoscl', 'immun', 'autistic', 'hand', 'axillary', 'loss', 'amniocentesis', 'cont', 'nonhemo', 'hypnic', 'indus', 'portal', 'leukocytopenia', 'engorgement', 'hyperparathyroidism', 'amblyopia', 'infec', 'cholecyst', 'attn', 'pop', 'triplets', 'behavr', 'metatarsal', 'diverticulum', 'spleen', 'subarachnoid', 'tbcco', 'angina', 'dacryolith', 'tk', 'peanut', 'encephalitis', 'inhibition', 'ureteric', 'single', 'anatmc', 'cond', 'hearing', 'viscera', 'ureth', 'nonrheum', 'mosquito', 'flutter', 'sampling', 'uri', 'nezelof', 'choledochlith', 'venereal', 'irrit', 'lg', 'situs', 'abducens', 'bedclothes', 'adjustment', 'wrst', 'manif', 'blind', 'hydantoin', 'rock', 'stricture', 'tiss', 'skis', 'pericarditis', 'caregiving', 'vesl', 'impacted', 'rbc', 'life', 'nutrition', 'salpingitis', 'choleperitonitis', 'parkinsonism', 'contraception', 'wll', 'vaccination', 'vena', 'nonfatal', 'arousal', 'dysmenorrhea', 'pyriform', 'dive', 'lingual', 'osteopathy', 'diaper', 'glauc', 'eggs', 'carbamates', 'psorospermiasis', 'bsl', 'inters', 'arts', 'insip', 'ascites', 'metacarpal', 'histoplasm', 'tors', 'sclera', 'cloudy', 'crtd', 'torpedo', 'arterial', 'antipruritics', 'public', 'esotropia', 'spon', 'transient', 'lewy', 'foreign', 'hypersom', 'aft', 'birthwt', 'fibromatoses', 'concentrate', 'vasa', 'bisphosphonates', 'analgesic', 'proteus', 'clitoris', 'insufficiency', 'gastrointest', 'munition', 'boydii', 'siwe', 'irregular', 'pros', 'duodenale', 'respiratory', 'foster', 'awareness', 'dissect', 'incompetence', 'gb', 'barbiturat', 'intrinsic', 'stph', 'defects', 'oddi', 'passengr', 'cramp', 'dermatophyt', 'underachievment', 'cephalosporin', 'obstr', 'sp', 'continua', 'astring', 'topical', 'opn', 'cran', 'asphyxia', 'suture', 'scotoma', 'bacteria', 'detachmnt', 'disabilty', 'alit', 'isoimm', 'spondylitis', 'glossodynia', 'mate', 'thyroiditis', 'hemplga', 'vssl', 'hemorrhagic', 'radiation', 'victm', 'syst', 'mansonella', 'scap', 'spasmodic', 'novel', 'tremor', 'sezary', 'rid', 'hypospadias', 'dmac', 'macrodactylia', 'gymnastics', 'thalassemia', 'strongyloidiasis', 'dyst', 'c', 'paraganglia', 'pneumonic', 'arizona', 'cystourethral', 'myelopathy', 'saphenous', 'cranial', 'strict', 'playgrnd', 'subinvolutn', 'rnd', 'nerves', 'hypersen', 'caused', 'bladder', 'purulent', 'keyboarding', 'antihyperten', 'feil', 'urethr', 'interfrnce', 'clostridium', 'springboard', 'cocidioidomycosis', 'lcl', 'oropharyngeal', 'intrcap', 'shel', 'milk', 'un', 'nephrolithias', 'anxiolytc', 'bse', 'nds', 'tinnitus', 'absnc', 'enterobiasis', 'euthyroid', 'protozoal', 'vsl', 'hypoxemia', 'immunosup', 'thromboangiit', 'hematemesis', 'autoimmun', 'jejunum', 'hyperlipidemia', 'sep', 'electroencephalogram', 'dsrpt', 'both', 'altitud', 'conjunctivit', 'atypical', 'antacid', 'lymphocytic', 'replacmnt', 'compart', 'carpometacarpal', 'flaccid', 'enterocele', 'aerial', 'progr', 'delays', 'scndhnd', 'incl', 'brach', 'near', 'nephrogen', 'phalan', 'shearing', 'cataract', 'displacmnt', 'mycosis', 'hyperpigmentation', 'mastoiditis', 'aniridia', 'origin', 'prosthes', 'hallucinogens', 'intestinal', 'trichomon', 'jaundice', 'gastrtis', 'hypernasality', 'dislocate', 'placental', 'mopping', 'study', 'neuroma', 'grease', 'norwalk', 'pauciart', 'stereotypic', 'interstit', 'disrp', 'postcatheter', 'antipsychotic', 'unarmed', 'unemployment', 'inguin', 'horizontl', 'transplnt', 'dislocation', 'concretions', 'gallstone', 'tibiofibula', 'fragile', 'nephrosonephritis', 'dvc', 'athetoid', 'motn', 'hypocalcemia', 'limited', 'fungoides', 'kaposi', 'cerebrum', 'clavic', 'tran', 'hiatus', 'brdrlne', 'genitalia', 'motion', 'clincal', 'ulna', 'ren', 'ehrlichiosis', 'cracked', 'pneumohemothor', 'mycotox', 'thrombot', 'cervic', 'cephl', 'anticonvul', 'distortion', 'male', 'bursal', 'purp', 'stren', 'chromosomal', 'bordetella', 'depth', 'pg', 'remov', 'plastic', 'entrhmrg', 'commode', 'brown', 'void', 'huntington', 'erythromycin', 'nocturia', 'folwg', 'contr', 'anodontia', 'prescrip', 'hypoglycemic', 'occipital', 'esoph', 'pannus', 'dysfunctions', 'dysp', 'convalescen', 'penis', 'wrng', 'chlamydial', 'thor', 'ancylostoma', 'phal', 'plast', 'pyeloneph', 'hyperemia', 'mesen', 'bulbar', 'retinitis', 'curb', 'procr', 'pathologic', 'carotid', 'aff', 'poor', 'orofacial', 'pyem', 'oxazolidine', 'infusn', 'blizzard', 'expsure', 'shift', 'hear', 'rate', 'sporotrichosis', 'e', 'atelectasis', 'heat', 'vater', 'osteochondrit', 'cerumen', 'impulsiveness', 'exertion', 'diastolc', 'hindgut', 'parapsoriasis', 'capsulit', 'circumst', 'srcma', 'isolat', 'failure', 'thyrotox', 'gram', 'ischemic', 'tetralogy', 'n', 'pupillary', 'axilla', 'neoplasms', 'gastroparesis', 'secretion', 'tool', 'mechanical', 'barrett', 'shouldr', 'ftl', 'malig', 'compliance', 'alveolar', 'epilepsy', 'ghost', 'defic', 'strain', 'connec', 'rheumatic', 'fevers', 'bag', 'uranium', 'disrup', 'embryon', 'trematode', 'ulcerative', 'bike', 'combat', 'prblms', 'caustic', 'bovine', 'retract', 'preauricular', 'introverted', 'instantaneous', 'cns', 'extradural', 'disproportion', 'antidepres', 'osteoarthrosis', 'organs', 'rectal', 'tibia', 'press', 'purine', 'lumbago', 'adhesions', 'pn', 'refusal', 'conjunctivochalasis', 'betting', 'dstrb', 'vis', 'tonic', 'cellulitis', 'hypersensit', 'mycoses', 'inoculat', 'decreased', 'antidiab', 'scalp', 'microphthalmos', 'transposition', 'epispadias', 'glass', 'bl', 'scrot', 'abscess', 'leucoma', 'valgum', 'open', 'handcrafts', 'aspirat', 'cm', 'joint', 'sts', 'analysis', 'histiocytosis', 'acetonuria', 'phys', 'unknwn', 'hyperact', 'arachnd', 'hx', 'schmorls', 'myopia', 'hematom', 'tick', 'examination', 'exotropia', 'mgkrcyt', 'osteoarthros', 'anes', 'lactat', 'esotrop', 'limb', 'fallot', 'drawn', 'anm', 'prsc', 'dermatophytosis', 'incmplet', 'spontaneous', 'campylobacter', 'combinations', 'cortex', 'homicidal', 'monofixation', 'aortic', 'urachus', 'occlu', 'chronic', 'solar', 'spider', 'anisometropia', 'sensoneur', 'esophagus', 'nephrosis', 'thrombophlebit', 'trait', 'skating', 'exercises', 'met', 'past', 'tics', 'influenae', 'mineralcorticoid', 'toler', 'bromine', 'habits', 'amin', 'domnt', 'infection', 'sed', 'development', 'roller', 'cleanser', 'envir', 'vn', 'superior', 'test', 'ntrc', 'rltd', 'conjunc', 'surveill', 'stable', 'ilium', 'nephroptosis', 'ani', 'club', 'chldhd', 'estrogen', 'thyrtox', 'bullet', 'dsdr', 'mixed', 'dispropor', 'genit', 'nail', 'calcaneovalgus', 'expert', 'hypoinsulinemia', 'diplopia', 'narcotic', 'actinic', 'sk', 'throm', 'entrpath', 'sensitization', 'taste', 'specfied', 'monkey', 'multip', 'bed', 'trnsmt', 'avulsion', 'mine', 'sphinc', 'polyuria', 'aftcre', 'cushing', 'pectus', 'ileus', 'aplastic', 'cognitive', 'conditn', 'bulimia', 'tricusp', 'monkeypox', 'managemt', 'alphafetoprotein', 'gastrojejunal', 'capslr', 'mcr', 'vehicle', 'contraceptive', 'hematuria', 'tp', 'rosea', 'pars', 'monocytosis', 'cachexia', 'sudden', 'hyprplsia', 'subcondyl', 'hemoglobinopath', 'tarsometatarsal', 'marfan', 'carbohydr', 'synechiae', 'mv', 'kleptomania', 'gingivostomat', 'coarctation', 'antifung', 'creutzfldt', 'x', 'werdnig', 'heav', 'lk', 'geographic', 'iris', 'airway', 'anovulation', 'dendritic', 'skn', 'undet', 'periodontosis', 'rsult', 'unspcfied', 'wart', 'hysteria', 'surgery', 'emotion', 'humoral', 'scars', 'quads', 'central', 'postcardiac', 'jump', 'schizo', 'idiopath', 'odontoclasia', 'pseudopterygium', 'forest', 'anaphylaxis', 'whitlow', 'antifungal', 'pituitary', 'appar', 'gendr', 'polycythemia', 'lacer', 'peristalsis', 'homonymous', 'pump', 'gastroptosis', 'remission', 'append', 'ossificans', 'remision', 'leiomyoma', 'seg', 'mucous', 'hr', 'hypn', 'obstruc', 'herpes', 'genetc', 'midfoot', 'abor', 'alkalizing', 'neurogenic', 'neur', 'infc', 'pro', 'vent', 'ejaculation', 'entrnvsv', 'necrobacillosis', 'polo', 'indu', 'cntrcptv', 'connect', 'ainhum', 'sqm', 'ray', 'retinopathy', 'biol', 'grnlma', 'venezuelan', 'bilry', 'halothane', 'interstitial', 'cytlogy', 'ds', 'cognf', 'replac', 'jaws', 'ing']


In [72]:
stop_list=alphabet_low+stopwords

In [73]:
### define python -> spark sql function to remove stop words and find description words
def stopword_remove(word_list):
    cleaned=[w for w in word_list if not w in stop_list if len(w)>2]
    code_descriptions=[w for w in cleaned if  w in idc9_word_list ]
    return code_descriptions
spark_stop_remove = psf.udf(lambda text: stopword_remove(text), ArrayType(StringType()))   


## clean to just word 

In [74]:
note_admit_clean=note_admit_label.select('*',spark_cleaner(psf.col("TEXT")).alias("TEXT_clean")).cache()
#.limit(1).take(1)[0]['<lambda>(TEXT)']

In [75]:
tokenizer = Tokenizer(inputCol="TEXT_clean", outputCol="tokens")
note_admit_token = tokenizer.transform(note_admit_clean).cache()

In [76]:
# run to cache
note_admit_token.count()

51113

In [77]:
# remove stops 
note_admit_stop = note_admit_token.select('*',spark_stop_remove(psf.col("tokens")).alias("tokens_clean")).cache()

In [78]:
note_admit_stop.select('tokens_clean').take(1)

[Row(tokens_clean=['date', 'discharge', 'date', 'date', 'birth', 'sex', 'medicine', 'patient', 'drugs', 'complaint', 'nausea', 'major', 'surgical', 'procedure', 'central', 'history', 'present', 'illness', 'old', 'ischemic', 'severe', 'type', 'nausea', 'hypotension', 'heart', 'failure', 'clinic', 'kidneys', 'severe', 'renal', 'artery', 'orthopnea', 'vomiting', 'diarrhea', 'recent', 'use', 'decreased', 'initial', 'insulin', 'blood', 'urine', 'central', 'past', 'medical', 'history', 'ischemic', 'severe', 'type', 'blindness', 'complete', 'occlusion', 'cri', 'renal', 'social', 'history', 'alone', 'smoking', 'alcohol', 'use', 'family', 'history', 'non', 'physical', 'exam', 'grade', 'sem', 'diastolic', 'around', 'iliac', 'fossa', 'evidence', 'abscess', 'right', 'lung', 'base', 'pneumonia', 'atelectasis', 'clinical', 'bilateral', 'pleural', 'right', 'left', 'stomach', 'gastroparesis', 'vascular', 'transplant', 'kidney', 'right', 'lower', 'brief', 'course', 'old', 'type', 'history', 'vessel', '

In [79]:
note_admit_stop.columns

['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'NEXT_ADMISSION', 'NEXT_ADMISSION_TYPE', 'days_next_admit', 'CATEGORY', 'TEXT', 'row_number', 'label', 'TEXT_clean', 'tokens', 'tokens_clean']

In [82]:
#remove lists and columns not fed to create features
note_admit_stop_reduce=note_admit_stop.select(note_admit_stop.columns[:10]+['tokens_clean','label']).cache()

In [ ]:
note_admit_stop_reduce.count()

In [ ]:
note_admit_stop.write.parquet('wasb:///final_idc9_filt.parquet')

# if starting from here 
# use 
>'final_idc9_filt' 

# else use 

>'note_admit_stop'

In [6]:
final_idc9_filt = spark.read.parquet('wasb:///final_idc9_filt.parquet')


In [7]:
#write parquet this is the base 
final_idc9_filt.repartition(numPartitions=1).write.parquet('wasb:///final_tokens_with_text.parquet')

In [7]:
final_idc9_filt.columns

['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'NEXT_ADMISSION', 'NEXT_ADMISSION_TYPE', 'days_next_admit', 'CATEGORY', 'TEXT', 'row_number', 'label', 'TEXT_clean', 'tokens', 'tokens_clean']

In [8]:
# if from scratch use 'note_admit_stop'
final_data=final_idc9_filt.select(final_idc9_filt.columns[:10]+['tokens_clean','label']).cache()

In [10]:
#keep
# final_data.repartition(numPartitions=1).write.parquet('wasb:///final_data_V2.parquet')

# vectorize 

In [9]:
import pyspark.ml.feature 
from pyspark.ml.feature import CountVectorizer

In [ ]:
# fill = psf.array().cast("array<string>")
# topics_a = psf.coalesce(col("tokens"), fill)
# note_admit_stop.withColumn("tokens", topics_a)

In [10]:
token_vector = CountVectorizer(inputCol="tokens_clean", outputCol="features")

In [11]:
#create sparse vectors
final_data2=token_vector.fit(final_data).transform(final_data).cache()


In [35]:
# final_data.columns[:9]

In [15]:
final_data2.columns

['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'NEXT_ADMISSION', 'NEXT_ADMISSION_TYPE', 'days_next_admit', 'CATEGORY', 'tokens_clean', 'label', 'features']

In [16]:
final_data_for_model=final_data2

In [ ]:
#2178 features

In [19]:
final_data_for_model.take(1)

[Row(HADM_ID=198608, SUBJECT_ID=231, ADMITTIME=datetime.datetime(2176, 5, 14, 21, 12), DISCHTIME=datetime.datetime(2176, 6, 5, 15, 0), DEATHTIME=None, ADMISSION_TYPE='EMERGENCY', NEXT_ADMISSION=datetime.datetime(2178, 8, 6, 14, 31), NEXT_ADMISSION_TYPE='EMERGENCY', days_next_admit=792, CATEGORY='Discharge summary', tokens_clean=['date', 'discharge', 'date', 'date', 'birth', 'sex', 'history', 'present', 'illness', 'old', 'male', 'hiv', 'fevers', 'chills', 'five', 'weeks', 'progressive', 'shortness', 'breath', 'cough', 'two', 'three', 'weeks', 'cough', 'sputum', 'patient', 'sore', 'throat', 'nausea', 'vomiting', 'one', 'clinic', 'short', 'course', 'symptoms', 'patient', 'headache', 'neck', 'stiffness', 'chest', 'pain', 'orthopnea', 'edema', 'abdominal', 'pain', 'diarrhea', 'dysuria', 'penile', 'discharge', 'blood', 'urine', 'stool', 'ten', 'weight', 'loss', 'five', 'weeks', 'new', 'rash', 'chest', 'non', 'pruritic', 'past', 'medical', 'history', 'human', 'virus', 'viral', 'patient', 'pas

In [94]:
#write
# final_data_for_model.repartition(numPartitions=1).write.parquet('wasb:///Data_for_modeling.parquet')

In [76]:
# final_data.write.parquet('wasb:///final_data.parquet')

In [81]:
# final_data.select(final_data.columns[:10]+['features']).write.parquet('wasb:///final_data_filtered.parquet')

In [95]:
final_data_for_model.columns

['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'NEXT_ADMISSION', 'NEXT_ADMISSION_TYPE', 'days_next_admit', 'CATEGORY', 'tokens_clean', 'label', 'features']

# models

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LinearSVC,NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [14]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lsvc = LinearSVC(maxIter=10, regParam=0.1)
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

In [17]:
(trainingData, testData) = final_data_for_model.randomSplit([0.7, 0.3])

In [18]:
#fit model 
model_lr=lr.fit(trainingData)


In [19]:
model_lsvc=lsvc.fit(trainingData)

In [20]:
model_nb=nb.fit(trainingData)

In [29]:
#params for CV
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

In [30]:
##cross val 
crossval_lr = CrossValidator(estimator=LogisticRegression(),
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5) 

In [31]:
cvModel_lr = crossval_lr.fit(trainingData)

In [32]:
predictions_cvlr =cvModel_lr.transform(testData)

In [21]:
#transforms back to spark data frame with predicyion data
predictions_lr =model_lr.transform(testData)
# predictions_cvlr =cvModel_lr.transform(testData)

predictions_lsvc =model_lsvc.transform(testData)
predictions_nb =model_nb.transform(testData)



In [22]:
columns_to_select=['HADM_ID', 'SUBJECT_ID','label','rawPrediction','probability','prediction']

An error occurred while calling o408.csv.
: java.lang.UnsupportedOperationException: CSV data source does not support struct<type:tinyint,size:int,indices:array<int>,values:array<double>> data type.
	at org.apache.spark.sql.execution.datasources.csv.CSVUtils$.org$apache$spark$sql$execution$datasources$csv$CSVUtils$$verifyType$1(CSVUtils.scala:127)
	at org.apache.spark.sql.execution.datasources.csv.CSVUtils$$anonfun$verifySchema$1.apply(CSVUtils.scala:131)
	at org.apache.spark.sql.execution.datasources.csv.CSVUtils$$anonfun$verifySchema$1.apply(CSVUtils.scala:131)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
	at scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
	at org.apache.spark.sql.types.StructType.foreach(StructType.scala:99)
	at org.apache.spark.sql.execution.datasources.csv.CSVUtils$.verifySchema(CSVUtils.scala:131)
	at org.apache.spark.sql.execution.datasources.csv.CSVFileForma

In [26]:
columns_to_select=['HADM_ID', 'SUBJECT_ID','label','rawPrediction','prediction']

In [33]:
predictions_cvlr.select(columns_to_select).repartition(numPartitions=1).write.parquet('wasb:///cross_val_logreg_pred.csv')

In [27]:
# columns_to_select=['HADM_ID', 'SUBJECT_ID','label','rawPrediction','prediction']

predictions_cvlr.select(columns_to_select).write.csv('wasb:///cross_val_logreg_pred.csv')
predictions_lsvc.select(columns_to_select).repartition(numPartitions=1).write.parquet('wasb:///support_v_pred.parquet')
predictions_nb.select(columns_to_select).repartition(numPartitions=1).write.parquet('wasb:///naive_bayes_pred.parquet')
predictions_lr.select(columns_to_select).repartition(numPartitions=1).write.parquet('wasb:///log_reg_pred.parquet')
# (numPartitions=1).write.parquet('wasb:///final_data_V2.parquet')


In [78]:
predictions_nb.()

['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'NEXT_ADMISSION', 'NEXT_ADMISSION_TYPE', 'days_next_admit', 'CATEGORY', 'tokens_clean', 'label', 'features', 'rawPrediction', 'probability', 'prediction']

In [74]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
### one for each model or cv
accuracy_lr = evaluator.evaluate(predictions_lr.select('label','prediction'))
accuracy_lscv = evaluator.evaluate(predictions_lsvc.select('label','prediction'))
accuracy_nb = evaluator.evaluate(predictions_nb.select('label','prediction'))



In [75]:
accuracy_cvlr = evaluator.evaluate(predictions_cvlr.select('label','prediction'))

In [76]:
accuracy_lr , accuracy_lscv, accuracy_nb

(0.9381214024071167, 0.9380559916274201, 0.7216117216117216)

In [77]:
accuracy_cvlr

0.9094060701203558